# Stocks Research Agent

This notebook defines a set of tools (skills) and tries it for a sample agent

# 0) IMPORTS

In [11]:
# Setup autoreload - automatically reload modules when they change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Verify environment setup
import os
import sys

print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"\nEnvironment variables loaded:")
print(f"OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"POLYGON_API_KEY: {'✓ Set' if os.getenv('POLYGON_API_KEY') else '✗ Not set'}")


Python version: 3.12.7 (main, Oct  1 2024, 02:05:46) [Clang 16.0.0 (clang-1600.0.26.3)]
Python executable: /Users/realmistic/Documents/stocks-scoring-agent/.venv/bin/python

Environment variables loaded:
OPENAI_API_KEY: ✓ Set
POLYGON_API_KEY: ✓ Set


In [13]:
# Import key libraries
import yfinance as yf
import pandas as pd
from openai import OpenAI

from pprint import pprint

print("Libraries imported successfully!")

Libraries imported successfully!


# 1) AGENT TOOLS

## 1.0 Ticker (Stock) Info - > many useful stats, incl. some of the important ratios

In [14]:
from typing import Any

def get_company_info(ticker: str) -> dict[str, Any]:
    """
    Get comprehensive company information and fundamental data for a stock ticker.
    
    Returns key metrics organized by category:
    - Company basics: website, industry, sector, employees, officers
    - Price data: current, previous close, day range, 52-week range
    - Market metrics: market cap, volume, beta, PE ratios
    - Valuation: margins, book value, price ratios
    - Ownership: insider/institutional holdings, short interest
    - Analyst data: EPS estimates, targets, recommendations
    - Financial health: cash, returns, growth rates
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')
    
    Returns:
        Dictionary with ticker and organized company information
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        info = ticker_obj.get_info()

        if not info:
            return {"ticker": ticker, "error": "No company info available"}

        # Define the key fields to extract (organized by category)
        key_fields = {
            # Company basics
            "company": ["website", "industry", "sector", "longBusinessSummary",
                        "fullTimeEmployees", "companyOfficers", "region", "fullExchangeName"],

            # Price data
            "price": ["currentPrice", "previousClose", "open", "dayLow", "dayHigh",
                    "regularMarketDayRange", "fiftyTwoWeekLow", "fiftyTwoWeekHigh",
                    "fiftyTwoWeekRange", "allTimeHigh", "allTimeLow"],

            # Market metrics
            "market": ["marketCap", "volume", "averageVolume", "averageVolume10days",
                    "beta", "trailingPE", "forwardPE", "trailingPegRatio"],

            # Moving averages
            "averages": ["fiftyDayAverage", "twoHundredDayAverage",
                        "fiftyDayAverageChange", "twoHundredDayAverageChange"],

            # Valuation ratios
            "valuation": ["priceToSalesTrailing12Months", "priceToBook", "bookValue",
                        "profitMargins", "grossMargins", "ebitdaMargins", "operatingMargins"],

            # Ownership & short interest
            "ownership": ["sharesOutstanding", "floatShares", "sharesPercentSharesOut",
                        "heldPercentInsiders", "heldPercentInstitutions",
                        "sharesShort", "shortRatio", "shortPercentOfFloat"],

            # EPS & earnings
            "earnings": ["trailingEps", "forwardEps", "earningsQuarterlyGrowth",
                        "earningsGrowth", "revenueGrowth", "epsTrailingTwelveMonths",
                        "epsForward", "epsCurrentYear"],

            # Analyst targets & recommendations
            "analyst": ["targetHighPrice", "targetLowPrice", "targetMeanPrice",
                        "targetMedianPrice", "recommendationMean", "recommendationKey",
                        "numberOfAnalystOpinions", "averageAnalystRating"],

            # Financial health
            "financial": ["totalCash", "totalCashPerShare", "totalDebt", "totalRevenue",
                        "freeCashflow", "operatingCashflow", "returnOnAssets",
                        "returnOnEquity", "debtToEquity", "currentRatio", "quickRatio"]
        }

        # Extract data by category
        result = {"ticker": ticker}

        for category, fields in key_fields.items():
            category_data = {}
            for field in fields:
                if field in info:
                    category_data[field] = info[field]
            if category_data:
                result[category] = category_data

        return result

    except Exception as e:
        return {"ticker": ticker, "error": f"Failed to get company info: {str(e)}"}

In [15]:

# Get all company info
info = get_company_info("TSLA")
pprint(info)


{'analyst': {'averageAnalystRating': '2.6 - Hold',
             'numberOfAnalystOpinions': 40,
             'recommendationKey': 'hold',
             'recommendationMean': 2.63043,
             'targetHighPrice': 600.0,
             'targetLowPrice': 120.0,
             'targetMeanPrice': 401.401,
             'targetMedianPrice': 432.0},
 'averages': {'fiftyDayAverage': 444.9456,
              'fiftyDayAverageChange': -11.202789,
              'twoHundredDayAverage': 362.26364,
              'twoHundredDayAverageChange': 71.47916},
 'company': {'companyOfficers': [{'age': 54,
                                  'exercisedValue': 0,
                                  'fiscalYear': 2024,
                                  'maxAge': 1,
                                  'name': 'Mr. Elon R. Musk',
                                  'title': 'Co-Founder, Technoking of Tesla, '
                                           'CEO & Director',
                                  'unexercisedValue': 0,
 

In [16]:

# Access specific categories
if "error" not in info:
    # Company basics
    print("\n=== Company Info ===")
    print(f"Industry: {info['company']['industry']}")
    print(f"Sector: {info['company']['sector']}")
    print(f"Employees: {info['company']['fullTimeEmployees']:,}")
    print(f"Website: {info['company']['website']}")

    # Current price & targets
    print("\n=== Price & Targets ===")
    print(f"Current: ${info['price']['currentPrice']:.2f}")
    print(f"52-Week Range: {info['price']['fiftyTwoWeekRange']}")
    print(f"Analyst Target (Mean): ${info['analyst']['targetMeanPrice']:.2f}")
    print(f"Recommendation: {info['analyst']['recommendationKey']}")

    # Key metrics
    print("\n=== Key Metrics ===")
    print(f"Market Cap: ${info['market']['marketCap']:,.0f}")
    print(f"Forward P/E: {info['market']['forwardPE']:.2f}")
    print(f"Profit Margin: {info['valuation']['profitMargins']:.2%}")

    # Ownership
    print("\n=== Ownership ===")
    print(f"Insider: {info['ownership']['heldPercentInsiders']:.2%}")
    print(f"Institutional: {info['ownership']['heldPercentInstitutions']:.2%}")
    print(f"Short Interest: {info['ownership']['shortRatio']}")

    # Growth
    print("\n=== Growth ===")
    print(f"Revenue Growth: {info['earnings']['revenueGrowth']:.2%}")
    print(f"Earnings Growth: {info['earnings']['earningsGrowth']:.2%}")

# Convert a category to DataFrame
if "analyst" in info:
    df = pd.DataFrame([info["analyst"]])
    print(df)



=== Company Info ===
Industry: Auto Manufacturers
Sector: Consumer Cyclical
Employees: 125,665
Website: https://www.tesla.com

=== Price & Targets ===
Current: $433.73
52-Week Range: 214.25 - 498.83
Analyst Target (Mean): $401.40
Recommendation: hold

=== Key Metrics ===
Market Cap: $1,442,550,120,448
Forward P/E: 196.81
Profit Margin: 5.31%

=== Ownership ===
Insider: 12.56%
Institutional: 49.93%
Short Interest: 0.97

=== Growth ===
Revenue Growth: 11.60%
Earnings Growth: -37.10%
   targetHighPrice  targetLowPrice  targetMeanPrice  targetMedianPrice  \
0            600.0           120.0          401.401              432.0   

   recommendationMean recommendationKey  numberOfAnalystOpinions  \
0             2.63043              hold                       40   

  averageAnalystRating  
0           2.6 - Hold  


## 1.1 EPS trend

In [79]:
from typing import Any

def get_eps_trend(ticker: str) -> dict[str, Any]:
    """
    Get the EPS (Earnings Per Share) trend for a given stock ticker - showing how analyst consensus has changed over time for different periods (quarterly, yearly)
    and diffent points in the past (current, 7daysAgo, 30daysAgo, etc.).
    Index: 0q (This Quarter),  +1q (Next Quarter),  0y (This Year),  +1y (Next Year) 
    and columns showing estimates from different points in the past (current, 7daysAgo, 30daysAgo, etc.). 

    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')
    
    Returns:
        Dictionary with ticker and EPS trend data
    """

    try:
        ticker_obj = yf.Ticker(ticker)
        result = ticker_obj.get_eps_trend()

        if isinstance(result, pd.DataFrame):
            if result.empty:
                return {"ticker": ticker, "error": "No EPS trend data available"}
            result['period']=result.index # create a new column 'period' from the index
            return {"ticker": ticker, "data": result.to_dict(orient='records')}

        # Fallback: wrap unexpected types
        if isinstance(result, dict):
            return {"ticker": ticker, "data": [result]}

        raise TypeError(f"Unexpected return type from get_eps_trend: {type(result)}")

    except Exception as e:
        return {"ticker": ticker, "error": f"Failed to get EPS trend: {str(e)}"}

In [83]:
r = get_eps_trend("TSLA")
# raw output to be used by an agent
print(r)
# pretty output for human consumption
print(f"\nFormatted output for ticker {r['ticker']}:")
print(pd.DataFrame(r['data']))

{'ticker': 'TSLA', 'data': [{'current': 0.44048, '7daysAgo': 0.44578, '30daysAgo': 0.4488, '60daysAgo': 0.43919, '90daysAgo': 0.46771, 'period': '0q'}, {'current': 0.43644, '7daysAgo': 0.43644, '30daysAgo': 0.44611, '60daysAgo': 0.44611, '90daysAgo': 0.45942, 'period': '+1q'}, {'current': 1.63333, '7daysAgo': 1.63796, '30daysAgo': 1.64119, '60daysAgo': 1.64634, '90daysAgo': 1.68606, 'period': '0y'}, {'current': 2.20383, '7daysAgo': 2.21713, '30daysAgo': 2.25452, '60daysAgo': 2.23864, '90daysAgo': 2.37572, 'period': '+1y'}]}

Formatted output for ticker TSLA:
   current  7daysAgo  30daysAgo  60daysAgo  90daysAgo period
0  0.44048   0.44578    0.44880    0.43919    0.46771     0q
1  0.43644   0.43644    0.44611    0.44611    0.45942    +1q
2  1.63333   1.63796    1.64119    1.64634    1.68606     0y
3  2.20383   2.21713    2.25452    2.23864    2.37572    +1y


## 1.2 Earnings dates, previous EPS actual vs. predicted and surprise; next expected earnings date

In [ ]:
from typing import Any

def get_earnings_dates(ticker: str) -> dict[str, Any]:
    """
    Get earnings call dates for a stock ticker.
    
    Returns historical earnings data including:
    - Expected EPS
    - Actual EPS  
    - Surprise percentage
    - Earnings dates from multiple quarters and years
    - Next earnings call date
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')

    Returns:
        Dictionary with ticker and earnings dates data, surprise (%) - how reported earnings compared to expectations
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        result = ticker_obj.get_earnings_dates()

        if isinstance(result, pd.DataFrame):
            if result.empty:
                return {"ticker": ticker, "error": "No earnings data available"}

            # Normalize common datetime-like columns to yyyy-mm-dd strings
            result.index = pd.to_datetime(result.index, errors="coerce").strftime("%Y-%m-%d")
            
            # Include the index (dates) as a column before converting to dict
            result = result.reset_index().rename(columns={"index": "date"})
            return {"ticker": ticker, "data": result.to_dict(orient='records')}

        # Unexpected type fallback
        raise TypeError(f"Unexpected return type from get_earnings_dates: {type(result)}")

    except Exception as e:
        return {"ticker": ticker, "error": f"Failed to get earnings dates: {str(e)}"}

In [97]:
r = get_earnings_dates("TSLA")
# raw output to be used by an agent
print(r)
# pretty output for human consumption
print(f"\nFormatted output for ticker {r['ticker']}:")
print(pd.DataFrame(r['data']))

{'ticker': 'TSLA', 'data': [{'Earnings Date': '2026-01-28', 'EPS Estimate': 0.44, 'Reported EPS': nan, 'Surprise(%)': nan}, {'Earnings Date': '2025-10-22', 'EPS Estimate': 0.44, 'Reported EPS': 0.39, 'Surprise(%)': -12.16}, {'Earnings Date': '2025-07-23', 'EPS Estimate': 0.3, 'Reported EPS': 0.33, 'Surprise(%)': 10.49}, {'Earnings Date': '2025-04-22', 'EPS Estimate': 0.35, 'Reported EPS': 0.12, 'Surprise(%)': -65.25}, {'Earnings Date': '2025-01-29', 'EPS Estimate': 0.77, 'Reported EPS': 0.73, 'Surprise(%)': -4.83}, {'Earnings Date': '2024-10-23', 'EPS Estimate': 0.5, 'Reported EPS': 0.62, 'Surprise(%)': 24.76}, {'Earnings Date': '2024-07-23', 'EPS Estimate': 0.62, 'Reported EPS': 0.52, 'Surprise(%)': -16.15}, {'Earnings Date': '2024-04-23', 'EPS Estimate': 0.49, 'Reported EPS': 0.45, 'Surprise(%)': -8.14}, {'Earnings Date': '2024-01-24', 'EPS Estimate': 0.74, 'Reported EPS': 0.71, 'Surprise(%)': -4.1}, {'Earnings Date': '2023-10-18', 'EPS Estimate': 0.73, 'Reported EPS': 0.66, 'Surpris

## 1.3 More of EPS/Growth stats and Analysts estimates

In [22]:
from typing import Any

def get_earnings_analysis(ticker: str) -> dict[str, Any]:
    """
    Get comprehensive earnings and EPS analysis data for a stock ticker.
    
    Combines multiple analyst data sources:
    1. Earnings Estimates - Consensus EPS estimates (avg, low, high, year-ago, analyst count)
    2. EPS Revisions - How analysts have revised estimates (up/down last 7/30 days)
    3. Growth Estimates - Expected earnings growth vs index benchmark
    4. Earnings History - Historical actual vs estimated EPS with surprise %
    
    Period notation:
    - 0q: Current fiscal quarter
    - +1q: Next fiscal quarter
    - 0y: Current fiscal year
    - +1y: Next fiscal year
    - LTG: Long-term growth (growth estimates only)
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')
    
    Returns:
        Dictionary with ticker and all earnings analysis data
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        result = {
            "ticker": ticker,
            "earnings_estimates": None,
            "eps_revisions": None,
            "growth_estimates": None,
            "earnings_history": None
        }

        # 1. Get earnings estimates
        try:
            earnings_est = ticker_obj.get_earnings_estimate()
            if isinstance(earnings_est, pd.DataFrame) and not earnings_est.empty:
                earnings_est = earnings_est.reset_index().rename(columns={"index": "period"})
                result["earnings_estimates"] = earnings_est.to_dict(orient='records')
        except Exception as e:
            result["earnings_estimates"] = {"error": str(e)}

        # 2. Get EPS revisions
        try:
            eps_rev = ticker_obj.get_eps_revisions()
            if isinstance(eps_rev, pd.DataFrame) and not eps_rev.empty:
                eps_rev = eps_rev.reset_index().rename(columns={"index": "period"})
                result["eps_revisions"] = eps_rev.to_dict(orient='records')
        except Exception as e:
            result["eps_revisions"] = {"error": str(e)}

        # 3. Get growth estimates
        try:
            growth_est = ticker_obj.get_growth_estimates()
            if isinstance(growth_est, pd.DataFrame) and not growth_est.empty:
                growth_est = growth_est.reset_index().rename(columns={"index": "period"})
                result["growth_estimates"] = growth_est.to_dict(orient='records')
        except Exception as e:
            result["growth_estimates"] = {"error": str(e)}

        # 4. Get earnings history
        try:
            earnings_hist = ticker_obj.get_earnings_history()
            if isinstance(earnings_hist, pd.DataFrame) and not earnings_hist.empty:
                earnings_hist = earnings_hist.reset_index().rename(columns={"index": "quarter"})
                result["earnings_history"] = earnings_hist.to_dict(orient='records')
        except Exception as e:
            result["earnings_history"] = {"error": str(e)}

        # Check if we got any data at all
        has_data = any(
            result[key] is not None and not isinstance(result[key], dict) or (isinstance(result[key], dict) and "error" not in result[key])
            for key in ["earnings_estimates", "eps_revisions", "growth_estimates", "earnings_history"]
        )

        if not has_data:
            return {"ticker": ticker, "error": "No earnings analysis data available"}

        return result

    except Exception as e:
        return {"ticker": ticker, "error": f"Failed to get earnings analysis: {str(e)}"}


In [23]:

# Get comprehensive earnings analysis
analysis = get_earnings_analysis("TSLA")
pprint(analysis)


{'earnings_estimates': [{'avg': 0.44048,
                         'growth': -0.3966,
                         'high': 0.65,
                         'low': 0.14,
                         'numberOfAnalysts': 25,
                         'period': '0q',
                         'yearAgoEps': 0.73},
                        {'avg': 0.43644,
                         'growth': 0.6164,
                         'high': 0.57,
                         'low': 0.27725,
                         'numberOfAnalysts': 12,
                         'period': '+1q',
                         'yearAgoEps': 0.27},
                        {'avg': 1.63333,
                         'growth': -0.32509997,
                         'high': 2.03,
                         'low': 1.14846,
                         'numberOfAnalysts': 34,
                         'period': '0y',
                         'yearAgoEps': 2.42},
                        {'avg': 2.20383,
                         'growth': 0.3493,
            

## 1.4 Daily historical prices (Open, High, Low, Close, Volume)

In [24]:
from typing import Any

def get_historical_prices(ticker: str, period: str = "2y", interval: str = "1d") -> dict[str, Any]:
    """
    Get historical price data for a stock ticker.
    
    Returns OHLCV (Open, High, Low, Close, Volume) data for the specified period.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')
        period: Time period to fetch (default: "2y")
                Valid periods: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
        interval: Data interval (default: "1d")
                Valid intervals: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
    
    Returns:
        Dictionary with ticker and historical price data
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        result = ticker_obj.history(period=period, interval=interval)

        if isinstance(result, pd.DataFrame):
            if result.empty:
                return {"ticker": ticker, "error": f"No historical data available for {ticker}"}

            # Convert datetime index to date strings for cleaner output
            result = result.copy()
            result.index = result.index.date
            result = result.reset_index().rename(columns={"index": "Date"})

            return {
                "ticker": ticker,
                "period": period,
                "interval": interval,
                "data": result.to_dict(orient='records')
            }

        raise TypeError(f"Unexpected return type from history: {type(result)}")

    except Exception as e:
        return {"ticker": ticker, "error": f"Failed to get historical prices: {str(e)}"}


In [25]:
from pprint import pprint
# Get 2 years of daily data (default)
prices = get_historical_prices("TSLA")
pprint(prices)

{'data': [{'Close': 240.4499969482422,
           'Date': datetime.date(2024, 1, 8),
           'Dividends': 0.0,
           'High': 241.25,
           'Low': 235.3000030517578,
           'Open': 236.13999938964844,
           'Stock Splits': 0.0,
           'Volume': 85166600},
          {'Close': 234.9600067138672,
           'Date': datetime.date(2024, 1, 9),
           'Dividends': 0.0,
           'High': 238.9600067138672,
           'Low': 232.0399932861328,
           'Open': 238.11000061035156,
           'Stock Splits': 0.0,
           'Volume': 96705700},
          {'Close': 233.94000244140625,
           'Date': datetime.date(2024, 1, 10),
           'Dividends': 0.0,
           'High': 235.5,
           'Low': 231.2899932861328,
           'Open': 235.10000610351562,
           'Stock Splits': 0.0,
           'Volume': 91628500},
          {'Close': 227.22000122070312,
           'Date': datetime.date(2024, 1, 11),
           'Dividends': 0.0,
           'High': 230.929992

## 1.5 Ticker news (Yahoo Finance)

In [26]:
from typing import Any

def get_ticker_news(ticker: str, count: int = 10) -> dict[str, Any]:
    """
    Get recent news articles for a stock ticker from Yahoo Finance.
    
    Returns key information including title, description, summary, publication date,
    and canonical URL for each news article.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')
        count: Maximum number of news articles to return (default: 10)
    
    Returns:
        Dictionary with ticker and news data
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        news_raw = ticker_obj.get_news()

        if not news_raw:
            return {"ticker": ticker, "error": "No news available"}

        # Convert to DataFrame to extract content
        news_df = pd.DataFrame(news_raw)

        if 'content' not in news_df.columns:
            return {"ticker": ticker, "error": "Unexpected news data format"}

        # Extract content and convert to DataFrame
        content_df = pd.DataFrame(news_df['content'].tolist())

        # Select only the important fields
        important_fields = ['title', 'description', 'summary', 'pubDate', 'canonicalUrl']

        # Filter to only existing fields
        available_fields = [f for f in important_fields if f in content_df.columns]
        news_filtered = content_df[available_fields]

        # Limit to requested count
        news_filtered = news_filtered.head(count)

        return {
            "ticker": ticker,
            "count": len(news_filtered),
            "data": news_filtered.to_dict(orient='records')
        }

    except Exception as e:
        return {"ticker": ticker, "error": f"Failed to get news: {str(e)}"}


In [27]:

# Get 10 most recent news articles (default)
news = get_ticker_news("TSLA", count=2)
pprint(news)

{'count': 2,
 'data': [{'canonicalUrl': {'lang': 'en-US',
                            'region': 'US',
                            'site': 'finance',
                            'url': 'https://finance.yahoo.com/video/ceo-shake-ups-2026-why-213152226.html'},
           'description': '<p>From pay packages to succession, investors have '
                          'been keeping a close eye on CEOs in 2025.</p>\n'
                          '<p>Harvard University executive education fellow '
                          'and former Medtronic (<a data-i13n="cpos:1;pos:1" '
                          'href="https://finance.yahoo.com/quote/MDT">MDT</a>) '
                          'CEO Bill George joins Market Domination to discuss '
                          "the CEOs he's watching in 2026.</p>\n"
                          '<p>To watch more expert insights and analysis on '
                          'the latest market action, check out more <a '
                          'data-i13n="cpos:2;pos:1"

## 1.6 Ticker news macro (Massive.com (ex. Polygon.io) - 5000 recent news + minsearch RAG)

In [28]:
from typing import Any
import requests
import pandas as pd
import os
from datetime import datetime, timezone
from minsearch import Index
from tqdm import tqdm

# Global variable to store the news index (built once, reused)
_news_index = None
_news_documents = None

def build_polygon_news_index(api_calls: int = 5, news_per_call: int = 1000) -> dict[str, Any]:
    """
    Fetch news from Polygon.io (Massive.com) and build a searchable index.
    
    This should be called once to fetch and index news. The index is stored
    globally and reused by search functions.
    
    Args:
        api_calls: Number of API calls to make (default: 5, fetches ~5000 articles)
        news_per_call: Number of news articles per API call (max: 1000)
    
    Returns:
        Dictionary with status and article count
    """
    global _news_index, _news_documents

    try:
        api_key = os.getenv('POLYGON_API_KEY')
        if not api_key:
            return {"error": "POLYGON_API_KEY not found in environment"}

        now = datetime.now(timezone.utc).strftime("%Y-%m-%d")
        all_news = None
        max_date = now

        print(f"Fetching {api_calls * news_per_call} news articles...")

        for i in tqdm(range(api_calls), desc="API calls"):
            url = f"https://api.massive.com/v2/reference/news?order=desc&limit={news_per_call}&sort=published_utc&published_utc.lt={max_date}&apiKey={api_key}"

            try:
                r = requests.get(url, timeout=10)
                r.raise_for_status()
                data = r.json()

                if 'results' not in data:
                    print(f"No 'results' in response. Keys: {data.keys()}")
                    continue

                cur = pd.json_normalize(data['results'])

                if all_news is None:
                    all_news = cur
                else:
                    all_news = pd.concat([all_news, cur], ignore_index=True)

                max_date = cur.published_utc.min()

            except requests.exceptions.RequestException as e:
                print(f"API call {i+1} failed: {e}")
                continue

        if all_news is None or all_news.empty:
            return {"error": "Failed to fetch news articles"}

        # Convert to documents
        _news_documents = all_news.to_dict(orient='records')

        # Preprocess documents
        print("Preprocessing documents...")
        for doc in tqdm(_news_documents, desc="Converting fields"):
            if isinstance(doc.get('tickers'), list):
                doc['tickers'] = ', '.join(doc['tickers'])
            if isinstance(doc.get('keywords'), list):
                doc['keywords'] = ', '.join(doc['keywords'])

            for field in ['title', 'description', 'author']:
                if doc.get(field) is None:
                    doc[field] = ''
                elif not isinstance(doc.get(field), str):
                    doc[field] = str(doc[field])

        # Build index
        print("Building search index...")
        _news_index = Index(
            text_fields=["title", "description", "keywords", "author", "tickers"],
            keyword_fields=["published_utc", "publisher.name"]
        )
        _news_index.fit(_news_documents)

        return {
            "status": "success",
            "articles_indexed": len(_news_documents),
            "message": f"Index built with {len(_news_documents)} articles"
        }

    except Exception as e:
        return {"error": f"Failed to build news index: {str(e)}"}


In [29]:
def search_news_by_ticker(ticker: str, num_results: int = 30) -> dict[str, Any]:
    """
    Search indexed news articles for a specific stock ticker.
    
    Searches across title, description, keywords, and tickers with boosting
    that prioritizes ticker matches.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'TSLA', 'AAPL', 'GOOGL')
        num_results: Maximum number of results to return (default: 30)
    
    Returns:
        Dictionary with ticker and matching news articles
    """
    global _news_index

    if _news_index is None:
        return {
            "ticker": ticker,
            "error": "News index not built. Call build_polygon_news_index() first."
        }

    try:
        results = _news_index.search(
            query=ticker,
            num_results=num_results,
            boost_dict={
                'tickers': 5.0,      # Highest boost for ticker field
                'title': 3.0,        # High boost for title
                'description': 1.5,  # Medium boost for description
                'keywords': 1.0      # Standard boost for keywords
            }
        )

        return {
            "ticker": ticker,
            "count": len(results),
            "data": results
        }

    except Exception as e:
        return {"ticker": ticker, "error": f"Search failed: {str(e)}"}


def search_news_by_query(query: str, num_results: int = 30) -> dict[str, Any]:
    """
    Search indexed news articles by free-text query.
    
    Searches across title, description, keywords, and tickers with boosting
    that prioritizes description and keyword matches.
    
    Args:
        query: Search query (e.g., 'Tesla competitors EV market', 'AI robotics')
        num_results: Maximum number of results to return (default: 30)
    
    Returns:
        Dictionary with query and matching news articles
    """
    global _news_index

    if _news_index is None:
        return {
            "query": query,
            "error": "News index not built. Call build_polygon_news_index() first."
        }

    try:
        results = _news_index.search(
            query=query,
            num_results=num_results,
            boost_dict={
                'tickers': 1.0,       # Standard boost for ticker field
                'title': 3.0,         # High boost for title
                'description': 5.0,   # Highest boost for description
                'keywords': 5.0       # Highest boost for keywords
            }
        )

        return {
            "query": query,
            "count": len(results),
            "data": results
        }

    except Exception as e:
        return {"query": query, "error": f"Search failed: {str(e)}"}


In [30]:
# USAGE EXAMPLE
# Step 1: Build the index once (takes a few minutes)
result = build_polygon_news_index(api_calls=5)
pprint(result)



Fetching 5000 news articles...


API calls: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it]


Preprocessing documents...


Converting fields: 100%|██████████| 5000/5000 [00:00<00:00, 1833014.60it/s]

Building search index...
{'articles_indexed': 5000,
 'message': 'Index built with 5000 articles',
 'status': 'success'}


In [31]:
# Step 2: Search by ticker
tsla_news = search_news_by_ticker("TSLA", num_results=10)
print(f"\nFound {tsla_news['count']} articles for {tsla_news['ticker']}")
for i, article in enumerate(tsla_news['data'][:5], 1):
    print(f"{i}. {article['title']}")
    print(f"   Published: {article['published_utc']}")
    print(f"   Tickers: {article['tickers']}\n")


Found 10 articles for TSLA
1. Tesla Is Knocking on $500 Again—Here’s What It Means For January
   Published: 2025-12-23T19:10:00Z
   Tickers: TSLA

2. What's Going On With The Uptick In Tesla Stock?
   Published: 2025-12-12T19:33:35Z
   Tickers: TSLA

3. Tesla Is About to Report Fourth-Quarter Deliveries. The Number May Be Weak -- And Investors May Not Care
   Published: 2026-01-01T00:15:00Z
   Tickers: TSLA

4. 3 Reasons I Will Never Buy Dogecoin
   Published: 2025-12-23T13:20:00Z
   Tickers: TSLA

5. Tesla Faces a Reality Check as Subsidy Pullbacks Weigh on Core Auto Volumes
   Published: 2026-01-02T17:16:00Z
   Tickers: TSLA



In [32]:
# Step 3: Search by query
query = "Tesla competitors EV margins autonomy AI robotics"
results = search_news_by_query(query, num_results=25)
print(f"\nFound {results['count']} articles for query: '{results['query']}'")
for i, article in enumerate(results['data'][:5], 1):
    print(f"{i}. {article['title']}")
    print(f"   Tickers: {article['tickers']}\n")


Found 25 articles for query: 'Tesla competitors EV margins autonomy AI robotics'
1. Should You Buy Tesla While It's Below $500?
   Tickers: TSLA

2. Should Investors Buy Tesla Stock Before 2026?
   Tickers: TSLA

3. Tesla Hits Ceiling In China? 2025 Sales Slump Could Mark A First For EV Giant
   Tickers: TSLA, BYDDY

4. This Robotics ETF Is Poised for 400% Growth in the Next 10 Years
   Tickers: ROBT, PATH, SYM, NVDA

5. 1 Stock I'd Buy Before Tesla in 2026
   Tickers: TSLA, NVDA, GOOG, GOOGL, AVGO, AMD, META, AMZN, AAPL, MSFT



## 1.7 Macro stats (market cap) - 10k companies (several csvs to save and query) 

  Key features:
  - ✅ Loads and merges all 4 databases (10k+ companies)
  - ✅ Unified search across market cap, P/E, dividend, and operating margin
  - ✅ Pre-built filters for value and growth investing strategies
  - ✅ Cache all databases for fast repeated queries
  - ✅ Multiple search criteria can be combined
  - ✅ Country filtering


In [33]:
from typing import Any, Optional
import requests
import pandas as pd
from io import StringIO

# Global variables to cache the databases
_companies_marketcap_db = None
_companies_pe_db = None
_companies_dividend_db = None
_companies_margin_db = None
_unified_db = None

def load_all_companies_databases(force_refresh: bool = False) -> dict[str, Any]:
    """
    Download and load all company databases from companiesmarketcap.com.
    
    Loads 4 databases:
    1. Market Cap - Top companies by market capitalization
    2. P/E Ratio - Top companies by price-to-earnings ratio
    3. Dividend Yield - Top companies by dividend yield percentage
    4. Operating Margin - Top companies by operating margin percentage
    
    The databases are cached globally and merged by ticker symbol for unified searching.
    Use force_refresh=True to re-download.
    
    Args:
        force_refresh: If True, re-download all databases even if cached
    
    Returns:
        Dictionary with status and database info
    """
    global _companies_marketcap_db, _companies_pe_db, _companies_dividend_db
    global _companies_margin_db, _unified_db

    # Return cached data if available
    if _unified_db is not None and not force_refresh:
        df = pd.DataFrame(_unified_db)
        return {
            "status": "loaded_from_cache",
            "total_companies": len(_unified_db),
            "available_columns": list(df.columns),
            "message": f"All databases loaded from cache"
        }

    try:
        databases = {
            "marketcap": "https://companiesmarketcap.com/usd/?download=csv",
            "pe_ratio": "https://companiesmarketcap.com/top-companies-by-pe-ratio/?download=csv",
            "dividend": "https://companiesmarketcap.com/top-companies-by-dividend-yield/?download=csv",
            "margin": "https://companiesmarketcap.com/top-companies-by-operating-margin/?download=csv"
        }

        loaded = {}

        for name, url in databases.items():
            print(f"Downloading {name} database...")
            try:
                response = requests.get(url, timeout=30)
                response.raise_for_status()
                csv_data = StringIO(response.text)
                df = pd.read_csv(csv_data)
                loaded[name] = df
                print(f"✓ Loaded {len(df)} companies from {name}")
            except Exception as e:
                print(f"✗ Failed to load {name}: {e}")
                loaded[name] = None

        # Store individual databases
        _companies_marketcap_db = loaded["marketcap"]
        _companies_pe_db = loaded["pe_ratio"]
        _companies_dividend_db = loaded["dividend"]
        _companies_margin_db = loaded["margin"]

        # Merge all databases by Symbol for unified view
        print("\nMerging databases...")

        # Start with market cap as base
        unified = loaded["marketcap"].copy()

        # Merge P/E ratio - only keep pe_ratio_ttm column
        if loaded["pe_ratio"] is not None and 'pe_ratio_ttm' in loaded["pe_ratio"].columns:
            pe_df = loaded["pe_ratio"][['Symbol', 'pe_ratio_ttm']]
            unified = unified.merge(pe_df, on='Symbol', how='left')
            print(f"✓ Added P/E ratio column")

        # Merge Dividend yield - only keep dividend_yield_ttm column
        if loaded["dividend"] is not None and 'dividend_yield_ttm' in loaded["dividend"].columns:
            div_df = loaded["dividend"][['Symbol', 'dividend_yield_ttm']]
            # Convert percentage to decimal
            div_df['dividend_yield_ttm'] = div_df['dividend_yield_ttm'] / 100.0 
            unified = unified.merge(div_df, on='Symbol', how='left')
            print(f"✓ Added Dividend yield column")

        # Merge Operating margin - only keep operating_margin_ttm column
        if loaded["margin"] is not None and 'operating_margin_ttm' in loaded["margin"].columns:
            margin_df = loaded["margin"][['Symbol', 'operating_margin_ttm']]
            # Convert percentage to decimal
            margin_df['operating_margin_ttm'] = margin_df['operating_margin_ttm']/100.0
            unified = unified.merge(margin_df, on='Symbol', how='left')
            print(f"✓ Added Operating margin column")

        print(f"\nFinal columns: {list(unified.columns)}")

        _unified_db = unified.to_dict(orient='records')

        return {
            "status": "success",
            "databases_loaded": {
                "marketcap": len(loaded["marketcap"]) if loaded["marketcap"] is not None else 0,
                "pe_ratio": len(loaded["pe_ratio"]) if loaded["pe_ratio"] is not None else 0,
                "dividend": len(loaded["dividend"]) if loaded["dividend"] is not None else 0,
                "margin": len(loaded["margin"]) if loaded["margin"] is not None else 0
            },
            "total_companies": len(_unified_db),
            "available_columns": list(unified.columns),
            "message": f"All databases merged with {len(_unified_db)} unique companies"
        }

    except Exception as e:
        return {"error": f"Failed to load databases: {str(e)}"}


def get_available_columns() -> dict[str, Any]:
    """
    Get list of available columns in the unified database.
    
    Returns:
        Dictionary with available column names
    """
    global _unified_db

    if _unified_db is None:
        return {"error": "Database not loaded. Call load_all_companies_databases() first."}

    df = pd.DataFrame(_unified_db)
    return {
        "columns": list(df.columns),
        "total_columns": len(df.columns)
    }


def search_companies(
    query: Optional[str] = None,
    ticker: Optional[str] = None,
    min_market_cap: Optional[float] = None,
    max_market_cap: Optional[float] = None,
    min_pe: Optional[float] = None,
    max_pe: Optional[float] = None,
    min_dividend: Optional[float] = None,
    max_dividend: Optional[float] = None,
    min_margin: Optional[float] = None,
    max_margin: Optional[float] = None,
    country: Optional[str] = None,
    limit: int = 50
) -> dict[str, Any]:
    """
    Search companies across all databases with comprehensive filtering.
    
    Args:
        query: Search by company name (case-insensitive partial match)
        ticker: Search by exact ticker symbol
        min_market_cap: Minimum market cap in USD
        max_market_cap: Maximum market cap in USD
        min_pe: Minimum P/E ratio
        max_pe: Maximum P/E ratio
        min_dividend: Minimum dividend yield (%)
        max_dividend: Maximum dividend yield (%)
        min_margin: Minimum operating margin (%)
        max_margin: Maximum operating margin (%)
        country: Filter by country (e.g., 'USA', 'China')
        limit: Maximum number of results (default: 50)
    
    Returns:
        Dictionary with matching companies and all available metrics
    """
    global _unified_db

    if _unified_db is None:
        return {"error": "Database not loaded. Call load_all_companies_databases() first."}

    try:
        df = pd.DataFrame(_unified_db)

        # Apply filters
        if ticker:
            df = df[df['Symbol'].str.upper() == ticker.upper()]

        if query:
            df = df[df['Name'].str.contains(query, case=False, na=False)]

        if min_market_cap is not None:
            df = df[df['marketcap'] >= min_market_cap]

        if max_market_cap is not None:
            df = df[df['marketcap'] <= max_market_cap]

        if 'pe_ratio_ttm' in df.columns:
            if min_pe is not None:
                df = df[pd.to_numeric(df['pe_ratio_ttm'], errors='coerce') >= min_pe]
            if max_pe is not None:
                df = df[pd.to_numeric(df['pe_ratio_ttm'], errors='coerce') <= max_pe]

        if 'dividend_yield_ttm' in df.columns:
            if min_dividend is not None:
                df = df[pd.to_numeric(df['dividend_yield_ttm'], errors='coerce') >= min_dividend]
            if max_dividend is not None:
                df = df[pd.to_numeric(df['dividend_yield_ttm'], errors='coerce') <= max_dividend]

        if 'operating_margin_ttm' in df.columns:
            if min_margin is not None:
                df = df[pd.to_numeric(df['operating_margin_ttm'], errors='coerce') >= min_margin]
            if max_margin is not None:
                df = df[pd.to_numeric(df['operating_margin_ttm'], errors='coerce') <= max_margin]

        if country:
            df = df[df['country'].str.upper() == country.upper()]

        # Limit results
        df = df.head(limit)

        if df.empty:
            return {
                "count": 0,
                "message": "No companies found matching criteria"
            }

        return {
            "count": len(df),
            "data": df.to_dict(orient='records')
        }

    except Exception as e:
        return {"error": f"Search failed: {str(e)}"}


def get_top_value_companies(
    min_dividend: float = 2.0/100.0, # 2%
    max_pe: float = 25,
    min_margin: float = 10/100.0, # 10%
    min_market_cap: float = 1_000_000_000,
    limit: int = 50
) -> dict[str, Any]:
    """
    Find potential value companies based on fundamental criteria.
    
    Default criteria:
    - Dividend yield >= 2%
    - P/E ratio <= 25
    - Operating margin >= 10%
    - Market cap >= $1B
    
    Args:
        min_dividend: Minimum dividend yield %
        max_pe: Maximum P/E ratio
        min_margin: Minimum operating margin %
        min_market_cap: Minimum market cap
        limit: Maximum results
    
    Returns:
        Dictionary with companies meeting value criteria
    """
    return search_companies(
        min_dividend=min_dividend,
        max_pe=max_pe,
        min_margin=min_margin,
        min_market_cap=min_market_cap,
        limit=limit
    )


def get_top_growth_companies(
    min_margin: float = 20/100.0, # 20%
    max_pe: Optional[float] = None,
    min_market_cap: float = 1_000_000_000,
    limit: int = 50
) -> dict[str, Any]:
    """
    Find potential growth companies based on fundamental criteria.
    
    Default criteria:
    - Operating margin >= 20% (high profitability)
    - Market cap >= $1B
    
    Args:
        min_margin: Minimum operating margin %
        max_pe: Maximum P/E ratio (optional)
        min_market_cap: Minimum market cap
        limit: Maximum results
    
    Returns:
        Dictionary with companies meeting growth criteria
    """
    return search_companies(
        min_margin=min_margin,
        max_pe=max_pe,
        min_market_cap=min_market_cap,
        limit=limit
    )



In [34]:
# USAGE EXAMPLE

# Load databases
result = load_all_companies_databases(force_refresh=True)
pprint(result)

# Search for value stocks
value_stocks = get_top_value_companies(min_dividend=3.0/100.0, max_pe=20, min_margin=15/100.0, limit=25)
print(f"\nFound {value_stocks['count']} value stocks")

if "data" in value_stocks:
    df = pd.DataFrame(value_stocks['data'])
    print(df[['Name', 'Symbol', 'marketcap', 'pe_ratio_ttm', 'dividend_yield_ttm', 'operating_margin_ttm']])


✓ Loaded 10642 companies from marketcap
✓ Loaded 10642 companies from pe_ratio
✓ Loaded 10642 companies from dividend
✓ Loaded 10642 companies from margin

Merging databases...
✓ Added P/E ratio column
✓ Added Dividend yield column
✓ Added Operating margin column

Final columns: ['Rank', 'Name', 'Symbol', 'marketcap', 'price (USD)', 'country', 'pe_ratio_ttm', 'dividend_yield_ttm', 'operating_margin_ttm']
{'available_columns': ['Rank',
                       'Name',
                       'Symbol',
                       'marketcap',
                       'price (USD)',
                       'country',
                       'pe_ratio_ttm',
                       'dividend_yield_ttm',
                       'operating_margin_ttm'],
 'databases_loaded': {'dividend': 10642,
                      'margin': 10642,
                      'marketcap': 10642,
                      'pe_ratio': 10642},
 'message': 'All databases merged with 10642 unique companies',
 'status': 'success',
 'total

/var/folders/_6/616g1v7j04jdsf8gy64v_t640000gn/T/ipykernel_38638/3381459230.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  div_df['dividend_yield_ttm'] = div_df['dividend_yield_ttm'] / 100.0
/var/folders/_6/616g1v7j04jdsf8gy64v_t640000gn/T/ipykernel_38638/3381459230.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  margin_df['operating_margin_ttm'] = margin_df['operating_margin_ttm']/100.0


In [35]:
# RAW DATA in the combined database
# checking the unified database of al
df = pd.DataFrame(_unified_db)

In [36]:
df.head()

,Rank,Name,Symbol,marketcap,price (USD),country,pe_ratio_ttm,dividend_yield_ttm,operating_margin_ttm
0,1,NVIDIA,NVDA,4607185518592,189.230,United States,46.7241,0.021086,62.26
1,2,Alphabet (Google),GOOG,3889785208832,322.220,United States,31.7864,0.255247,39.54
2,3,Apple,AAPL,3862211330048,260.250,United States,35.0361,0.393025,31.57
3,4,Microsoft,MSFT,3595422662656,483.700,United States,34.5635,0.697658,43.44
4,5,Amazon,AMZN,2587620016128,242.055,United States,33.8689,0.000000,10.75


# 2) Building an agent 

## 2.1) Simple agent - no memory

In [37]:
# 
!uv  pip list | grep openai

Using Python 3.12.7 environment at: /Users/realmistic/Documents/stocks-scoring-agent/.venv
openai                    2.13.0
openai-agents             0.6.3


In [38]:
import agents
print(f"📦 Version: {agents.__version__}")

📦 Version: 0.6.3


In [39]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = await Runner.run(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

A function calls self,  
Echoes in logic's deep cave—  
Ends when base is met.


In [40]:
from agents import function_tool

# in case we need WebSearch 
from agents import WebSearchTool

In [41]:
stock_agent = Agent(
    name='stock_agent',
    instructions="you're a helful assistant. Try to use all defined tools before going into the WebSearchTool",
    model='gpt-4o-mini',
    tools=[ 
            # WebSearchTool(), 
           function_tool(get_company_info),
           function_tool(get_eps_trend),
           function_tool(get_earnings_dates),
           function_tool(get_earnings_analysis),
           function_tool(get_historical_prices),
           function_tool(get_ticker_news),
           function_tool(search_news_by_ticker),
           function_tool(search_news_by_query),
           function_tool(search_companies),
           function_tool(get_top_value_companies),
           function_tool(get_top_growth_companies),
           ] 
)

In [42]:
from agents import Runner

runner = Runner()

results = await runner.run(stock_agent, 
                           input="get me the eps trend for TSLA and the latest earnings call date and results, news sentiment")

In [43]:
print(results.final_output)

### EPS Trend for Tesla (TSLA)

Here’s the EPS trend for TSLA over different periods:

- **Current EPS**: 0.44048
- **EPS Estimates**:
  - **7 Days Ago**: 0.44578
  - **30 Days Ago**: 0.4488
  - **60 Days Ago**: 0.43919
  - **90 Days Ago**: 0.46771
  
- **Previous EPS Trends** (Further Periods):
  - **Current**: 0.43644
    - 7 Days Ago: 0.43644
    - 30 Days Ago: 0.44611
    - 60 Days Ago: 0.44611
    - 90 Days Ago: 0.45942
  - **Current**: 1.63333
    - 7 Days Ago: 1.63796
    - 30 Days Ago: 1.64119
    - 60 Days Ago: 1.64634
    - 90 Days Ago: 1.68606
  - **Current**: 2.20383
    - 7 Days Ago: 2.21713
    - 30 Days Ago: 2.25452
    - 60 Days Ago: 2.23864
    - 90 Days Ago: 2.37572

### Latest Earnings Call Dates and Results

Here are the latest earnings call results for TSLA:

1. **Next Earnings Date**: January 28, 2026
   - **EPS Estimate**: **0.44**
   - **Reported EPS**: N/A
   - **Surprise(%)**: N/A

2. **Latest Earnings Date**: October 22, 2025
   - **EPS Estimate**: **0.44**
 

In [44]:
results.raw_responses

[ModelResponse(output=[ResponseFunctionToolCall(arguments='{"ticker":"TSLA"}', call_id='call_1C2BP1KSwcB4lQN7TANM73lg', name='get_eps_trend', type='function_call', id='fc_08022bc0e7bbdd3c00695ec7b95b348193ac6ee0010f835f2d', status='completed'), ResponseFunctionToolCall(arguments='{"ticker":"TSLA"}', call_id='call_5cazvzIWM6vOB7M2Tk64AWR4', name='get_earnings_dates', type='function_call', id='fc_08022bc0e7bbdd3c00695ec7b9ee7c81939d7f941bd37b6f69', status='completed'), ResponseFunctionToolCall(arguments='{"ticker":"TSLA","count":10}', call_id='call_ObF9fnBBC27FrnhfvGkEKjcm', name='get_ticker_news', type='function_call', id='fc_08022bc0e7bbdd3c00695ec7ba3434819389d291272d12541c', status='completed')], usage=Usage(requests=1, input_tokens=590, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=72, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=662, request_usage_entries=[]), response_id='resp_08022bc0e7bbdd3c00695ec7b786a881938ae6659c774d30

In [45]:
results.new_items

[ToolCallItem(agent=Agent(name='stock_agent', handoff_description=None, tools=[FunctionTool(name='get_company_info', description='Get comprehensive company information and fundamental data for a stock ticker.\n\nReturns key metrics organized by category:\n- Company basics: website, industry, sector, employees, officers\n- Price data: current, previous close, day range, 52-week range\n- Market metrics: market cap, volume, beta, PE ratios\n- Valuation: margins, book value, price ratios\n- Ownership: insider/institutional holdings, short interest\n- Analyst data: EPS estimates, targets, recommendations\n- Financial health: cash, returns, growth rates', params_json_schema={'properties': {'ticker': {'description': "Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')", 'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'get_company_info_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>.

In [46]:
pprint(results.new_items[0])

ToolCallItem(agent=Agent(name='stock_agent',
                         handoff_description=None,
                         tools=[FunctionTool(name='get_company_info',
                                             description='Get comprehensive '
                                                         'company information '
                                                         'and fundamental data '
                                                         'for a stock ticker.\n'
                                                         '\n'
                                                         'Returns key metrics '
                                                         'organized by '
                                                         'category:\n'
                                                         '- Company basics: '
                                                         'website, industry, '
                                                         'sector, emplo

In [47]:
# DEBUG: print everything for inspection (5k lines)
# from pprint import pprint
# pprint(vars(results))

In [48]:
# Write a wrapper functions to extract tool calls, outputs, and final message
                                                                                                                                                                
def get_tool_calls(results):                                                                                                                                       
    """Extract just the tool calls"""                                                                                                                              
    calls = []                                                                                                                                                     
    for response in results.raw_responses:                                                                                                                         
        for output in response.output:                                                                                                                             
            if hasattr(output, 'name') and hasattr(output, 'arguments'):                                                                                           
                calls.append({                                                                                                                                     
                    'name': output.name,                                                                                                                           
                    'arguments': output.arguments                                                                                                                  
                })                                                                                                                                                 
    return calls                                                                                                                                                   
                                                                                                                                                                    
def get_tool_outputs(results):                                                                                                                                     
    """Extract tool outputs with their names"""                                                                                                                    
    # Get tool calls from raw_responses                                                                                                                            
    calls = []                                                                                                                                                     
    for response in results.raw_responses:                                                                                                                         
        for output in response.output:                                                                                                                             
            if hasattr(output, 'name'):                                                                                                                            
                calls.append(output.name)                                                                                                                          
                                                                                                                                                                    
    # Get outputs from new_items                                                                                                                                   
    outputs = []                                                                                                                                                   
    for item in results.new_items:                                                                                                                                 
        if item.type == 'tool_call_output_item':                                                                                                                   
            outputs.append(item.output)                                                                                                                            
                                                                                                                                                                    
    return list(zip(calls, outputs))                                                                                                                               
                                                                                                                                                                    
def get_final_message(results):                                                                                                                                    
    """Get the final text response"""                                                                                                                              
    # Method 1: Use final_output                                                                                                                                   
    if results.final_output:                                                                                                                                       
        return results.final_output                                                                                                                                
                                                                                                                                                                    
    # Method 2: Extract from raw_responses                                                                                                                         
    for response in results.raw_responses:                                                                                                                         
        for output in response.output:                                                                                                                             
            if hasattr(output, 'content'):                                                                                                                         
                for content in output.content:                                                                                                                     
                    if hasattr(content, 'text'):                                                                                                                   
                        return content.text                        

In [49]:
                                                                                                                                                                
# Usage:                                                                                                                                                           
calls = get_tool_calls(results)                                                                                                                                    
tool_outputs = get_tool_outputs(results)                                                                                                                           
final_msg = get_final_message(results)                                                                                                                             
                                                                                                                                                                    
print("=" * 70)                                                                                                                                                    
print("TOOL CALLS")                                                                                                                                                
print("=" * 70)                                                                                                                                                    
for i, call in enumerate(calls, 1):                                                                                                                                
    print(f"\n{i}. {call['name']}({call['arguments']})")                                                                                                           
                                                                                                                                                                    
print("\n" + "=" * 70)                                                                                                                                             
print("TOOL OUTPUTS")                                                                                                                                              
print("=" * 70)                                                                                                                                                    
for i, (name, output) in enumerate(tool_outputs, 1):                                                                                                               
    print(f"\n{i}. {name}")                                                                                                                                        
    print(f"   Output keys: {list(output.keys()) if isinstance(output, dict) else type(output)}")                                                                  
    if isinstance(output, dict) and 'ticker' in output:                                                                                                            
        print(f"   Ticker: {output['ticker']}")                                                                                                                    
        if 'data' in output:                                                                                                                                       
            data = output['data']                                                                                                                                  
            if isinstance(data, list):                                                                                                                             
                print(f"   Data items: {len(data)}")                                                                                                               
                                                                                                                                                                    
print("\n" + "=" * 70)                                                                                                                                             
print("FINAL MESSAGE")                                                                                                                                             
print("=" * 70)                                                                                                                                                    
print(f"\nLength: {len(final_msg) if final_msg else 0} characters")                                                                                                
if final_msg:                                                                                                                                                      
    # Print first 500 chars                                                                                                                                        
    print(f"\nPreview:\n{final_msg[:500]}...")              

TOOL CALLS

1. get_eps_trend({"ticker":"TSLA"})

2. get_earnings_dates({"ticker":"TSLA"})

3. get_ticker_news({"ticker":"TSLA","count":10})

TOOL OUTPUTS

1. get_eps_trend
   Output keys: ['ticker', 'data']
   Ticker: TSLA
   Data items: 4

2. get_earnings_dates
   Output keys: ['ticker', 'data']
   Ticker: TSLA
   Data items: 25

3. get_ticker_news
   Output keys: ['ticker', 'count', 'data']
   Ticker: TSLA
   Data items: 10

FINAL MESSAGE

Length: 2579 characters

Preview:
### EPS Trend for Tesla (TSLA)

Here’s the EPS trend for TSLA over different periods:

- **Current EPS**: 0.44048
- **EPS Estimates**:
  - **7 Days Ago**: 0.44578
  - **30 Days Ago**: 0.4488
  - **60 Days Ago**: 0.43919
  - **90 Days Ago**: 0.46771
  
- **Previous EPS Trends** (Further Periods):
  - **Current**: 0.43644
    - 7 Days Ago: 0.43644
    - 30 Days Ago: 0.44611
    - 60 Days Ago: 0.44611
    - 90 Days Ago: 0.45942
  - **Current**: 1.63333
    - 7 Days Ago: 1.63796
    - 30 Days Ago: 1...


In [50]:
                                                                                                                                                    
# Clean extraction                                                                                                                                                 
print("📋 TOOLS CALLED:")                                                                                                                                          
for resp in results.raw_responses:                                                                                                                                 
    for out in resp.output:                                                                                                                                        
        if hasattr(out, 'name'):                                                                                                                                   
            print(f"  • {out.name}({out.arguments})")                                                                                                              
                                                                                                                                                                    
print(f"\n📊 OUTPUTS: {len(results.new_items)} items")                                                                                                             
for item in results.new_items:                                                                                                                                     
    if item.type == 'tool_call_output_item':                                                                                                                       
        ticker = item.output.get('ticker', 'N/A')                                                                                                                  
        print(f"  • Ticker: {ticker}, Keys: {list(item.output.keys())}")                                                                                           
                                                                                                                                                                    
print(f"\n💬 FINAL OUTPUT: {len(results.final_output)} chars")                                                                                                     
print(results.final_output[:300] + "..." if len(results.final_output) > 300 else results.final_output)                                                             
           

📋 TOOLS CALLED:
  • get_eps_trend({"ticker":"TSLA"})
  • get_earnings_dates({"ticker":"TSLA"})
  • get_ticker_news({"ticker":"TSLA","count":10})

📊 OUTPUTS: 7 items
  • Ticker: TSLA, Keys: ['ticker', 'data']
  • Ticker: TSLA, Keys: ['ticker', 'data']
  • Ticker: TSLA, Keys: ['ticker', 'count', 'data']

💬 FINAL OUTPUT: 2579 chars
### EPS Trend for Tesla (TSLA)

Here’s the EPS trend for TSLA over different periods:

- **Current EPS**: 0.44048
- **EPS Estimates**:
  - **7 Days Ago**: 0.44578
  - **30 Days Ago**: 0.4488
  - **60 Days Ago**: 0.43919
  - **90 Days Ago**: 0.46771
  
- **Previous EPS Trends** (Further Periods):
  -...


## 2.2) Agent with a Memory (passing context - runResult)

In [51]:
# let's build a wrapper for calls AND PASS THE CONTEXT FOR NEXT QUESTIONS
                                                                                                                                                                
from typing import Optional                                                                                                                                        
from agents import Runner                                                                                                                                          
from agents.result import RunResult                                                                                                                                
from IPython.display import display, Markdown, HTML                                                                                                                
                                                                                                                                                                    
runner = Runner()                                                                                                                                                  
                                                                                                                                                                    
async def ask(agent, question: str, context: Optional[RunResult] = None) -> RunResult:                                                                             
    """Async ask with rich notebook output."""                                                                                                                     
                                                                                                                                                                    
    # Question header                                                                                                                                              
    display(HTML(f"<h3>❓ {question}</h3>"))                                                                                                                       
                                                                                                                                                                    
    # Run agent                                                                                                                                                    
    if context:                                                                                                                                                    
        results = await runner.run(agent, input=question, context=context.new_items)                                                                               
    else:                                                                                                                                                          
        results = await runner.run(agent, input=question)                                                                                                          
                                                                                                                                                                    
    # Show tools                                                                                                                                                   
    tools_html = "<div style='padding: 10px; border-radius: 5px;'>"                                                                     
    tools_html += "<b>🔧 Tools Called:</b><br>"                                                                                                                    
    tool_count = 0                                                                                                                                                 
    for resp in results.raw_responses:                                                                                                                             
        for out in resp.output:                                                                                                                                    
            if hasattr(out, 'name'):                                                                                                                               
                tool_count += 1                                                                                                                                    
                tools_html += f"&nbsp;&nbsp;{tool_count}. <code>{out.name}({out.arguments})</code><br>"                                                            
                                                                                                                                                                    
    if tool_count == 0:                                                                                                                                            
        tools_html += "&nbsp;&nbsp;<i>(No tools called)</i><br>"                                                                                                   
    tools_html += "</div>"                                                                                                                                         
                                                                                                                                                                    
    display(HTML(tools_html))                                                                                                                                      
                                                                                                                                                                    
    # Show response as markdown                                                                                                                                    
    display(Markdown("### 💬 Response:"))                                                                                                                          
    display(Markdown(results.final_output))                                                                                                                        
                                                                                                                                                                    
    return results                   

In [52]:
# Usage:
r1 = await ask(stock_agent, "What's TSLA's EPS trend and surprise historical?")                                                                                          

### 💬 Response:

### Tesla, Inc. (TSLA) EPS Trend

#### Current EPS Estimates:
- **Current Quarter (0q)**: 
  - **Average**: 0.44048
  - **Low**: 0.14
  - **High**: 0.65
  - **Year Ago EPS**: 0.73
  - **Analysts Count**: 25
  - **Growth**: -39.66%

- **Next Quarter (+1q)**: 
  - **Average**: 0.43644
  - **Low**: 0.27725
  - **High**: 0.57
  - **Year Ago EPS**: 0.27
  - **Analysts Count**: 12
  - **Growth**: 61.64%

- **Current Year (0y)**: 
  - **Average**: 1.63333
  - **Low**: 1.14846
  - **High**: 2.03
  - **Year Ago EPS**: 2.42
  - **Analysts Count**: 34
  - **Growth**: -32.51%

- **Next Year (+1y)**: 
  - **Average**: 2.20383
  - **Low**: 1.31
  - **High**: 3.48
  - **Year Ago EPS**: 1.63333
  - **Analysts Count**: 33
  - **Growth**: 34.93%

### EPS Revisions:
1. **Current Quarter (0q)**: 
   - **Up Last 7 Days**: 2
   - **Down Last 7 Days**: 2
   - **Down Last 30 Days**: 5

2. **Next Quarter (+1q)**: 
   - **Up Last 7 Days**: 2
   - **Down Last 7 Days**: 0
   - **Down Last 30 Days**: 2

3. **Current Year (0y)**:
   - **Up Last 7 Days**: 3
   - **Down Last 7 Days**: 3
   - **Down Last 30 Days**: 6

4. **Next Year (+1y)**:
   - **Up Last 7 Days**: 3
   - **Down Last 7 Days**: 3
   - **Down Last 30 Days**: 6

### Historical EPS Surprises:
| Quarter Ending     | Actual EPS | Estimated EPS | Difference | Surprise (%) |
|---------------------|------------|----------------|------------|--------------|
| 2024-12-31          | 0.73       | 0.76703        | -0.04      | -4.83%       |
| 2025-03-31          | 0.27       | 0.41468        | -0.14      | -34.89%      |
| 2025-06-30          | 0.40       | 0.40391        | -0.00      | -0.97%       |
| 2025-09-30          | 0.50       | 0.55885        | -0.06      | -10.53%      |

### Summary
The current EPS estimates show a decline in growth from the previous year, while next year's estimates indicate a potential recovery. Recent revisions reflect mixed sentiment among analysts, with a notable number of downgrades in the past month. The historical surprises highlight some misses against estimates.

In [53]:
r2 = await ask(stock_agent, "Find similar stocks to TSLA on market cap and P/E ratio", context=r1)                                                   
# r3 = await ask(stock_agent, "Show extended analysis of competitors and strategic positioning?", context=r2)      

### 💬 Response:

Here are some stocks similar to **Tesla (TSLA)** in terms of market capitalization and P/E ratio:

### Selected Similar Stocks

1. **NVIDIA (NVDA)**
   - **Market Cap:** $4.61 Trillion
   - **P/E Ratio:** 46.72
   - **Price:** $189.23
   - **Dividend Yield:** 2.11%
   - **Operating Margin:** 62.26%

2. **Alphabet (GOOG)**
   - **Market Cap:** $3.89 Trillion
   - **P/E Ratio:** 31.79
   - **Price:** $322.22
   - **Dividend Yield:** 0.26%
   - **Operating Margin:** 39.54%

3. **Apple (AAPL)**
   - **Market Cap:** $3.86 Trillion
   - **P/E Ratio:** 35.04
   - **Price:** $260.25
   - **Dividend Yield:** 0.39%
   - **Operating Margin:** 31.57%

4. **Microsoft (MSFT)**
   - **Market Cap:** $3.60 Trillion
   - **P/E Ratio:** 34.56
   - **Price:** $483.70
   - **Dividend Yield:** 0.70%
   - **Operating Margin:** 43.44%

5. **Amazon (AMZN)**
   - **Market Cap:** $2.59 Trillion
   - **P/E Ratio:** 33.87
   - **Price:** $242.06
   - **Dividend Yield:** 0.00%
   - **Operating Margin:** 10.75%

6. **TSMC (TSM)**
   - **Market Cap:** $1.66 Trillion
   - **P/E Ratio:** 46.88
   - **Price:** $319.74
   - **Dividend Yield:** 0.98%
   - **Operating Margin:** 48.57%

7. **Meta Platforms (META)**
   - **Market Cap:** $1.64 Trillion
   - **P/E Ratio:** 28.12
   - **Price:** $649.20
   - **Dividend Yield:** 0.32%
   - **Operating Margin:** 44.91%

8. **Saudi Aramco (2222.SR)**
   - **Market Cap:** $1.52 Trillion
   - **P/E Ratio:** 14.82
   - **Price:** $6.29
   - **Dividend Yield:** 5.62%
   - **Operating Margin:** 41.85%

9. **Berkshire Hathaway (BRK-B)**
   - **Market Cap:** $1.07 Trillion
   - **P/E Ratio:** 14.39
   - **Price:** $495.97
   - **Dividend Yield:** 0.00%
   - **Operating Margin:** 20.54%

10. **Walmart (WMT)**
    - **Market Cap:** $900.85 Billion
    - **P/E Ratio:** 39.69
    - **Price:** $112.99
    - **Dividend Yield:** 0.83%
    - **Operating Margin:** 3.86%

### Notes
- **Market Cap of TSLA:** Approximately $1.44 Trillion
- **P/E Ratio of TSLA:** Approximately 303.13

These stocks were selected based on their market capitalization proximity and P/E ratio comparisons. Let me know if you need more specific insights!

In [54]:
# anotehr example of a chat ("Just be explicit in your questions"):                                                                                                                              
r1 = await ask(stock_agent, "What's Google EPS trend vs. big 7 companies?")                                                                                                              

### 💬 Response:

Here's a comparison of the EPS trend for Google (GOOGL) versus the top 7 growth companies, which include NVIDIA, Apple, Microsoft, Amazon, TSMC, and Meta Platforms.

### Google (GOOGL) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 2.629      |
| 7 Days Ago  | 2.627      |
| 30 Days Ago | 2.624      |
| 60 Days Ago | 2.618      |
| 90 Days Ago | 2.548      |

### NVIDIA (NVDA) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 1.522      |
| 7 Days Ago  | 1.522      |
| 30 Days Ago | 1.521      |
| 60 Days Ago | 1.419      |
| 90 Days Ago | 1.409      |

### Apple (AAPL) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 2.666      |
| 7 Days Ago  | 2.667      |
| 30 Days Ago | 2.663      |
| 60 Days Ago | 2.663      |
| 90 Days Ago | 2.482      |

### Microsoft (MSFT) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 3.861      |
| 7 Days Ago  | 3.861      |
| 30 Days Ago | 3.858      |
| 60 Days Ago | 3.865      |
| 90 Days Ago | 3.805      |

### Amazon (AMZN) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 1.939      |
| 7 Days Ago  | 1.939      |
| 30 Days Ago | 1.935      |
| 60 Days Ago | 1.935      |
| 90 Days Ago | 1.844      |

### TSMC (TSM) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 2.919      |
| 7 Days Ago  | 2.860      |
| 30 Days Ago | 2.856      |
| 60 Days Ago | 2.854      |
| 90 Days Ago | 2.539      |

### Meta Platforms (META) EPS Trend
| Period      | EPS        |
|-------------|------------|
| Current     | 8.170      |
| 7 Days Ago  | 8.170      |
| 30 Days Ago | 8.171      |
| 60 Days Ago | 8.161      |
| 90 Days Ago | 8.072      |

### Summary
- Google (GOOGL) has a current EPS of **2.629**, which shows consistent growth over the past 90 days.
- NVIDIA's current EPS is **1.522**, while Apple’s is **2.666** and Microsoft’s is **3.861**.
- Amazon has a current EPS of **1.939**, TSMC's is **2.919**, and Meta's is notably high at **8.170**.

Let me know if you need further details!

In [55]:
r2 = await ask(stock_agent, "Compare BIG 7 companies on valuations, analysts sentiment, investors demand, and growth prospects")                                                                                             

### 💬 Response:

Here’s a detailed comparison of the BIG 7 companies focusing on valuations, analyst sentiment, investor demand, and growth prospects:

| Company    | Market Cap ($B) | Current Price | P/E Ratio | Forward P/E | Analyst Rating | Analyst Target Range ($) | Revenue Growth (%) | Earnings Growth (%) | Profit Margin (%) |
|------------|-------------------|----------------|-----------|-------------|-----------------|-----------------------|---------------------|---------------------|--------------------|
| **Apple (AAPL)**    | 3,864.59            | 260.43         | 34.95     | 28.44       | Buy             | 215 - 350             | 7.90                | 9.12                | 26.92              |
| **Amazon (AMZN)**   | 2,586.82            | 241.98         | 34.18     | 30.84       | Strong Buy      | 245 - 360             | 13.40               | 36.40               | 11.06              |
| **Alphabet (GOOGL)**| 3,893.68            | 321.47         | 31.77     | 28.72       | Strong Buy      | 185 - 432             | 15.90               | 35.30               | 32.23              |
| **Microsoft (MSFT)**| 3,595.05            | 483.67         | 34.37     | 25.81       | Strong Buy      | 483 - 730             | 18.40               | 12.70               | 35.71              |
| **Meta (FB)**       | 383.63              | 42.41          | N/A       | N/A         | Buy             | 77 - 152.5            | 17.20               | 8.70                | 24.05              |
| **Netflix (NFLX)**  | 383.63              | 90.54          | 37.88     | 27.92       | Buy             | 90 - 134              | 17.20               | 8.70                | 24.05              |
| **Tesla (TSLA)**    | 1,441.21            | 433.31         | 303.03    | 196.63      | Hold            | 120 - 600             | 11.60               | -37.10              | 5.31               |

### Analysis

#### Valuations:
1. **P/E Ratios**: Tesla's P/E is substantially higher due to its high growth expectations. Apple, Amazon, Alphabet, and Microsoft have relatively lower P/E ratios, suggesting they are perceived as more stable.
2. **Forward P/E**: Again, Tesla's forward P/E is extremely high, indicating expected slower earnings in the coming year. Meanwhile, Microsoft and Amazon have promising future growth reflected in their P/E ratios.

#### Analysts' Sentiment:
1. **Analyst Rating**: Most companies received “Buy” or “Strong Buy” ratings, indicating a positive sentiment among analysts, with Tesla getting a hold recommendation.
2. **Analyst Target Range**: There is significant potential upside in target price ranges for all companies, with Amazon, Alphabet, and Microsoft showing the most promising targets.

#### Investors' Demand:
1. **Market Cap**: Apple and Alphabet lead in market cap, reflecting strong investor demand and confidence in their business models.
2. **Profit Margins**: Alphabet has the highest profit margin, suggesting efficient cost management and pricing power, followed by Microsoft and Apple. Tesla, on the other hand, has much lower margins due to its high developmental costs and competition.

#### Growth Prospects:
1. **Revenue Growth**: Both Amazon and Alphabet exhibit strong revenue growth rates relative to others, showing effective market expansion strategies.
2. **Earnings Growth**: Positive growth rates in earnings suggest not only are revenues rising, but the efficiency of converting sales into profits is also improving across most companies.

This comprehensive view illustrates where each of these companies stands in terms of valuation, analyst sentiment, investor demand, and growth potential in the current market landscape.

In [56]:
r3 = await ask(stock_agent, "Show top 10 growth companies in biotech sector")  

### 💬 Response:

Here are the top 10 growth companies that meet the criteria for high operating margins in the biotech sector:

| Rank | Name                      | Symbol | Market Cap (USD) | Price (USD) | Country       | P/E Ratio (TTM) | Dividend Yield (TTM) | Operating Margin (TTM) |
|------|---------------------------|--------|-------------------|-------------|---------------|------------------|----------------------|------------------------|
| 1    | NVIDIA                     | NVDA   | 4,607,185,518,592 | 189.23      | United States | 46.72            | 2.11%                | 62.26%                 |
| 2    | Alphabet (Google)         | GOOG   | 3,889,785,208,832 | 322.22      | United States | 31.79            | 25.52%               | 39.54%                 |
| 3    | Apple                      | AAPL   | 3,862,211,330,048 | 260.25      | United States | 35.04            | 39.30%               | 31.57%                 |
| 4    | Microsoft                  | MSFT   | 3,595,422,662,656 | 483.70      | United States | 34.56            | 69.77%               | 43.44%                 |
| 5    | Amazon                     | AMZN   | 2,587,620,016,128 | 242.06      | United States | 33.87            | 0.00%                | 10.75%                 |
| 6    | TSMC                      | TSM    | 1,658,332,905,472 | 319.74      | Taiwan        | 46.88            | 0.98%                | 48.57%                 |
| 7    | Meta Platforms (Facebook) | META   | 1,636,326,572,032 | 649.20      | United States | 28.12            | 32.21%               | 44.91%                 |
| 8    | Broadcom                  | AVGO   | 1,634,648,850,432 | 344.77      | United States | 70.76            | 69.79%               | 40.60%                 |
| 9    | Saudi Aramco              | 2222.SR| 1,520,782,348,561 | 6.29        | Saudi Arabia  | 14.82            | 5.62%                | 41.85%                 |
| 10   | Tesla                     | TSLA   | 1,442,657,206,272 | 433.78      | United States | 265.08           | 0.00%                | 7.54%                  |

### Note:
- These companies exhibit high operating margins, which can be indicative of strong profitability and growth potential.
- The list contains major players in the tech sector, but may not encompass traditional biotech companies specifically. 

If you'd like more focused results on specific biotech companies, please let me know!

## 2.3) Agents with explicit context of a ticker (current_ticker) and a history

In [57]:
# this thing automa
class Conversation:                                                                                                                                                
    def __init__(self, agent, runner):                                                                                                                             
        self.agent = agent                                                                                                                                         
        self.runner = runner                                                                                                                                       
        self.history = []                                                                                                                                          
        self.current_ticker = None                                                                                                                                 
                                                                                                                                                                    
    async def ask(self, question: str) -> RunResult:                                                                                                               
        """Ask with automatic context and ticker tracking."""                                                                                                      
                                                                                                                                                                    
        # Extract ticker from question if present                                                                                                                  
        import re                                                                                                                                                  
        ticker_match = re.search(r'\b([A-Z]{1,5})\b', question)                                                                                                    
        if ticker_match:                                                                                                                                           
            self.current_ticker = ticker_match.group(1)                                                                                                            
                                                                                                                                                                    
        # Add ticker to question if we have context and it's not mentioned                                                                                         
        if self.current_ticker and self.current_ticker not in question.upper():                                                                                    
            question = f"{question} (for {self.current_ticker})"                                                                                                   
                                                                                                                                                                    
        print(f"\n{'='*70}")                                                                                                                                       
        print(f"❓ {question}")                                                                                                                                    
        if self.current_ticker:                                                                                                                                    
            print(f"🏷️  Ticker: {self.current_ticker}")                                                                                                           
        print('='*70)                                                                                                                                              
                                                                                                                                                                    
        # Run with context from previous question                                                                                                                  
        context = self.history[-1].new_items if self.history else None                                                                                             
                                                                                                                                                                    
        if context:                                                                                                                                                
            results = await self.runner.run(self.agent, input=question, context=context)                                                                           
        else:                                                                                                                                                      
            results = await self.runner.run(self.agent, input=question)                                                                                            
                                                                                                                                                                    
        # Show tools                                                                                                                                               
        print("\n🔧 Tools Called:")                                                                                                                                
        tool_count = 0                                                                                                                                             
        for resp in results.raw_responses:                                                                                                                         
            for out in resp.output:                                                                                                                                
                if hasattr(out, 'name'):                                                                                                                           
                    tool_count += 1                                                                                                                                
                    print(f"  {tool_count}. {out.name}({out.arguments})")                                                                                          
                                                                                                                                                                    
        if tool_count == 0:                                                                                                                                        
            print("  (No tools called)")                                                                                                                           
                                                                                                                                                                    
        # Extract ticker from results if not already set                                                                                                           
        if not self.current_ticker:                                                                                                                                
            for item in results.new_items:                                                                                                                         
                if item.type == 'tool_call_output_item' and isinstance(item.output, dict):                                                                         
                    if 'ticker' in item.output:                                                                                                                    
                        self.current_ticker = item.output['ticker']                                                                                                
                        break                                                                                                                                      
                                                                                                                                                                    
        # Show response                                                                                                                                            
        print(f"\n{'='*70}")                                                                                                                                       
        print("💬 Response:")                                                                                                                                      
        print('='*70)                                                                                                                                              
        print(results.final_output)                                                                                                                                
        print()                                                                                                                                                    
                                                                                                                                                                    
        self.history.append(results)                                                                                                                               
        return results                                                                                                                                             
                                                                                                                                                                    
    def reset(self):                                                                                                                                               
        """Clear conversation history."""                                                                                                                          
        self.history = []                                                                                                                                          
        self.current_ticker = None                                                                                                                                 
                                                                                                                                                                    


In [58]:
# Usage                                                                                                                                        
convo = Conversation(stock_agent, runner)   

await convo.ask("What's TSLA's EPS trend and surprise historical?")   


❓ What's TSLA's EPS trend and surprise historical?
🏷️  Ticker: TSLA

🔧 Tools Called:
  1. get_eps_trend({"ticker":"TSLA"})
  2. get_earnings_analysis({"ticker":"TSLA"})

💬 Response:
### TSLA EPS Trend and Surprise Historical

#### EPS Trend
Here’s a summary of the EPS estimates for Tesla (TSLA) over various time periods:

- **Current Quarter (0q)**
  - **Current Avg. EPS**: 0.44048
  - **7 Days Ago**: 0.44578
  - **30 Days Ago**: 0.4488
  - **60 Days Ago**: 0.43919
  - **90 Days Ago**: 0.46771

- **Next Quarter (+1q)**
  - **Current Avg. EPS**: 0.43644
  - **7 Days Ago**: 0.43644
  - **30 Days Ago**: 0.44611
  - **60 Days Ago**: 0.44611
  - **90 Days Ago**: 0.45942

- **Current Year (0y)**
  - **Current Avg. EPS**: 1.63333
  - **7 Days Ago**: 1.63796
  - **30 Days Ago**: 1.64119
  - **60 Days Ago**: 1.64634
  - **90 Days Ago**: 1.68606

- **Next Year (+1y)**
  - **Current Avg. EPS**: 2.20383
  - **7 Days Ago**: 2.21713
  - **30 Days Ago**: 2.25452
  - **60 Days Ago**: 2.23864
  - **90

RunResult(input="What's TSLA's EPS trend and surprise historical?", new_items=[ToolCallItem(agent=Agent(name='stock_agent', handoff_description=None, tools=[FunctionTool(name='get_company_info', description='Get comprehensive company information and fundamental data for a stock ticker.\n\nReturns key metrics organized by category:\n- Company basics: website, industry, sector, employees, officers\n- Price data: current, previous close, day range, 52-week range\n- Market metrics: market cap, volume, beta, PE ratios\n- Valuation: margins, book value, price ratios\n- Ownership: insider/institutional holdings, short interest\n- Analyst data: EPS estimates, targets, recommendations\n- Financial health: cash, returns, growth rates', params_json_schema={'properties': {'ticker': {'description': "Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')", 'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'get_company_info_args', 'type': 'object', 'additionalProperties': False}, o

In [59]:
await convo.ask("What about analysts predictions of growth vs. index?")                                                                                            


❓ What about analysts predictions of growth vs. index? (for TSLA)
🏷️  Ticker: TSLA

🔧 Tools Called:
  1. get_earnings_analysis({"ticker":"TSLA"})

💬 Response:
Here are the analysts' predictions regarding Tesla, Inc. (TSLA) compared to the index, broken down by different periods:

### Earnings Estimates and Growth
- **Current Quarter (0q)**:
  - Average EPS Estimate: **0.44048**
  - Growth Rate: **-39.66%** (vs. year-ago EPS of **0.73**)
  - Index Trend: **6.98%**

- **Next Quarter (+1q)**:
  - Average EPS Estimate: **0.43644**
  - Growth Rate: **61.64%** (vs. year-ago EPS of **0.27**)
  - Index Trend: **12.01%**

- **Current Year (0y)**:
  - Average EPS Estimate: **1.63333**
  - Growth Rate: **-32.51%** (vs. year-ago EPS of **2.42**)
  - Index Trend: **15.65%**

- **Next Year (+1y)**:
  - Average EPS Estimate: **2.20383**
  - Growth Rate: **34.93%** (vs. year-ago EPS of **1.63333**)
  - Index Trend: **14.69%**

### Earnings Revisions
- **Current Quarter (0q)**:
  - Upgrades in last 7 

RunResult(input='What about analysts predictions of growth vs. index? (for TSLA)', new_items=[ToolCallItem(agent=Agent(name='stock_agent', handoff_description=None, tools=[FunctionTool(name='get_company_info', description='Get comprehensive company information and fundamental data for a stock ticker.\n\nReturns key metrics organized by category:\n- Company basics: website, industry, sector, employees, officers\n- Price data: current, previous close, day range, 52-week range\n- Market metrics: market cap, volume, beta, PE ratios\n- Valuation: margins, book value, price ratios\n- Ownership: insider/institutional holdings, short interest\n- Analyst data: EPS estimates, targets, recommendations\n- Financial health: cash, returns, growth rates', params_json_schema={'properties': {'ticker': {'description': "Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')", 'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'get_company_info_args', 'type': 'object', 'additionalPropert

In [60]:
await convo.ask("Show extended analysis of competitors and strategic positioning?") 


❓ Show extended analysis of competitors and strategic positioning? (for TSLA)
🏷️  Ticker: TSLA


[non-fatal] Tracing: server error 503, retrying.



🔧 Tools Called:
  1. get_company_info({"ticker":"TSLA"})
  2. get_earnings_analysis({"ticker":"TSLA"})
  3. get_eps_trend({"ticker":"TSLA"})
  4. get_earnings_dates({"ticker":"TSLA"})
  5. get_ticker_news({"ticker":"TSLA","count":5})

💬 Response:
### Tesla, Inc. (TSLA): Company Overview and Strategic Positioning

**Company Information:**
- **Website:** [Tesla](https://www.tesla.com)
- **Industry:** Auto Manufacturers
- **Sector:** Consumer Cyclical
- **Headquarters:** Austin, Texas
- **Founded:** 2003
- **Employees:** 125,665

**Business Segments:**
1. **Automotive:** Designs, develops, manufactures, leases, and sells electric vehicles. Offers services such as financing, leasing, vehicle insurance, and sales of regulatory credits.
2. **Energy Generation and Storage:** Engages in the design, manufacture, sale, and leasing of solar energy generation and storage systems.

**Current Market Position:**
- **Market Capitalization:** $1.44 trillion
- **Current Stock Price:** $432.28
- **52-We

RunResult(input='Show extended analysis of competitors and strategic positioning? (for TSLA)', new_items=[ToolCallItem(agent=Agent(name='stock_agent', handoff_description=None, tools=[FunctionTool(name='get_company_info', description='Get comprehensive company information and fundamental data for a stock ticker.\n\nReturns key metrics organized by category:\n- Company basics: website, industry, sector, employees, officers\n- Price data: current, previous close, day range, 52-week range\n- Market metrics: market cap, volume, beta, PE ratios\n- Valuation: margins, book value, price ratios\n- Ownership: insider/institutional holdings, short interest\n- Analyst data: EPS estimates, targets, recommendations\n- Financial health: cash, returns, growth rates', params_json_schema={'properties': {'ticker': {'description': "Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')", 'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'get_company_info_args', 'type': 'object', 'addit

In [61]:
await convo.ask("How is it doing against car manufacturers?") 


❓ How is it doing against car manufacturers? (for TSLA)
🏷️  Ticker: TSLA

🔧 Tools Called:
  1. get_company_info({"ticker":"TSLA"})
  2. get_top_growth_companies({"min_margin":20,"max_pe":null,"min_market_cap":1000000000,"limit":5})
  3. get_top_value_companies({"min_dividend":2,"max_pe":25,"min_margin":10,"min_market_cap":1000000000,"limit":5})

💬 Response:
### Tesla Overview
- **Current Price:** $432.06
- **Market Cap:** $1.44 Trillion
- **Industry:** Auto Manufacturers
- **Employees:** ~125,665

**Business Segment Summary:**
- Automotive: Electric vehicles, regulatory credits, part sales, insurance services, and financing.
- Energy Generation and Storage: Solar energy products, energy storage solutions, and related services.

### Comparison Against Other Car Manufacturers

#### Top Growth Companies in the Market
While Tesla is pivotal in the electric vehicle sector, here are the top growth companies for context (not directly in auto manufacturing):

1. **NVIDIA (NVDA)**
   - Market 

RunResult(input='How is it doing against car manufacturers? (for TSLA)', new_items=[ToolCallItem(agent=Agent(name='stock_agent', handoff_description=None, tools=[FunctionTool(name='get_company_info', description='Get comprehensive company information and fundamental data for a stock ticker.\n\nReturns key metrics organized by category:\n- Company basics: website, industry, sector, employees, officers\n- Price data: current, previous close, day range, 52-week range\n- Market metrics: market cap, volume, beta, PE ratios\n- Valuation: margins, book value, price ratios\n- Ownership: insider/institutional holdings, short interest\n- Analyst data: EPS estimates, targets, recommendations\n- Financial health: cash, returns, growth rates', params_json_schema={'properties': {'ticker': {'description': "Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')", 'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'get_company_info_args', 'type': 'object', 'additionalProperties': Fals

In [62]:
await convo.ask("What about Google? Give me a comparative analysis similar to Tesla.") 


❓ What about Google? Give me a comparative analysis similar to Tesla. (for TSLA)
🏷️  Ticker: TSLA

🔧 Tools Called:
  1. get_company_info({"ticker":"GOOGL"})
  2. get_company_info({"ticker":"TSLA"})
  3. get_earnings_analysis({"ticker":"GOOGL"})
  4. get_earnings_analysis({"ticker":"TSLA"})
  5. get_eps_trend({"ticker":"GOOGL"})
  6. get_eps_trend({"ticker":"TSLA"})
  7. get_earnings_dates({"ticker":"GOOGL"})
  8. get_earnings_dates({"ticker":"TSLA"})

💬 Response:
### Comparative Analysis: Alphabet Inc. (GOOGL) vs. Tesla, Inc. (TSLA)

#### Company Overview

| Metric                     | Alphabet Inc. (GOOGL)                                          | Tesla, Inc. (TSLA)                                               |
|----------------------------|---------------------------------------------------------------|------------------------------------------------------------------|
| Industry                   | Internet Content & Information                                 | Auto Manufactur

RunResult(input='What about Google? Give me a comparative analysis similar to Tesla. (for TSLA)', new_items=[ToolCallItem(agent=Agent(name='stock_agent', handoff_description=None, tools=[FunctionTool(name='get_company_info', description='Get comprehensive company information and fundamental data for a stock ticker.\n\nReturns key metrics organized by category:\n- Company basics: website, industry, sector, employees, officers\n- Price data: current, previous close, day range, 52-week range\n- Market metrics: market cap, volume, beta, PE ratios\n- Valuation: margins, book value, price ratios\n- Ownership: insider/institutional holdings, short interest\n- Analyst data: EPS estimates, targets, recommendations\n- Financial health: cash, returns, growth rates', params_json_schema={'properties': {'ticker': {'description': "Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')", 'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'title': 'get_company_info_args', 'type': 'object', 'ad

## 2.4) Structured output about one stock 

In [63]:
                                                                                                                                                                
from pydantic import BaseModel, Field                                                                                                                               
from typing import Optional, List                                                                                                                                   
from datetime import datetime                                                                                                                                       
                                                                                                                                                                    
class NewsItem(BaseModel):                                                                                                                                          
    title: str                                                                                                                                                      
    summary: str                                                                                                                                                    
    sentiment: str  # positive, negative, neutral                                                                                                                   
    published: str                                                                                                                                                  
                                                                                                                                                                    
class CompetitorComparison(BaseModel):                                                                                                                              
    ticker: str                                                                                                                                                     
    name: str                                                                                                                                                       
    pe_ratio: Optional[float] = None                                                                                                                                
    market_cap: Optional[float] = None                                                                                                                              
                                                                                                                                                                    
class EPSTrend(BaseModel):                                                                                                                                          
    current_quarter: float                                                                                                                                          
    vs_market_index: str  # "outperforming", "underperforming", "inline"                                                                                            
    trend_direction: str  # "improving", "declining", "stable"                                                                                                      
    surprise_history: List[float]  # Last 4 quarters surprise %                                                                                                     
                                                                                                                                                                    
class AnalystSentiment(BaseModel):                                                                                                                                  
    eps_revisions_7d: dict  # {"up": 2, "down": 1}                                                                                                                  
    eps_revisions_30d: dict                                                                                                                                         
    trend: str  # "improving", "declining", "mixed"                                                                                                                 
    growth_vs_index: float  # percentage difference                                                                                                                 
                                                                                                                                                                    
class Valuation(BaseModel):                                                                                                                                         
    current_price: float                                                                                                                                            
    week_52_high: float                                                                                                                                             
    week_52_low: float                                                                                                                                              
    distance_from_high_pct: float                                                                                                                                   
    distance_from_low_pct: float                                                                                                                                    
    pe_ratio: Optional[float]                                                                                                                                       
    pe_vs_competitors_avg: Optional[float]                                                                                                                          
    valuation_summary: str  # "expensive", "fair", "cheap"                                                                                                          
                                                                                                                                                                    
class DemandAnalysis(BaseModel):                                                                                                                                    
    recent_volume_trend: str  # "increasing", "decreasing", "stable"                                                                                                
    price_momentum_3m: float  # % change                                                                                                                            
    price_momentum_6m: float                                                                                                                                        
    trading_activity: str  # "high", "normal", "low"                                                                                                                
                                                                                                                                                                    
class FutureTrends(BaseModel):                                                                                                                                      
    key_themes: List[str]  # Main topics from news                                                                                                                  
    sentiment_direction: str  # "bullish", "bearish", "neutral"                                                                                                     
    catalyst_events: List[str]  # Upcoming events                                                                                                                   
    risk_factors: List[str]                                                                                                                                         
                                                                                                                                                                    
class CompanyAnalysisReport(BaseModel):                                                                                                                             
    ticker: str                                                                                                                                                     
    company_name: str                                                                                                                                               
    analysis_date: str                                                                                                                                              
                                                                                                                                                                    
    # Core metrics                                                                                                                                                  
    eps_trend: EPSTrend                                                                                                                                             
    analyst_sentiment: AnalystSentiment                                                                                                                             
    valuation: Valuation                                                                                                                                            
    demand_analysis: DemandAnalysis                                                                                                                                 
                                                                                                                                                                    
    # News & sentiment                                                                                                                                              
    recent_news: List[NewsItem]                                                                                                                                     
    competitor_news: List[NewsItem]                                                                                                                                 
                                                                                                                                                                    
    # Competitive position                                                                                                                                          
    competitors: List[CompetitorComparison]                                                                                                                         
                                                                                                                                                                    
    # Forward looking                                                                                                                                               
    future_outlook: FutureTrends                                                                                                                                    
                                                                                                                                                                    
    # Summary                                                                                                                                                       
    executive_summary: str                                                                                                                                          
    investment_thesis: str  # "buy", "hold", "sell" with reasoning                                                                                                  
                                                                                                                                                                    
    class Config:                                                                                                                                                   
        json_schema_extra = {                                                                                                                                       
            "example": {                                                                                                                                            
                "ticker": "TSLA",                                                                                                                                   
                "company_name": "Tesla Inc.",                                                                                                                       
                "eps_trend": {                                                                                                                                      
                    "current_quarter": 0.45,                                                                                                                        
                    "vs_market_index": "underperforming",                                                                                                           
                    "trend_direction": "declining"                                                                                                                  
                }                                                                                                                                                   
            }                                                                                                                                                       
        }    

/var/folders/_6/616g1v7j04jdsf8gy64v_t640000gn/T/ipykernel_38638/144847831.py:51: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class CompanyAnalysisReport(BaseModel):


In [64]:
                                                                                                                                                                
from agents import Agent, function_tool, WebSearchTool                                                                                                           
from typing import List                                                                                                                                             
                                                                                                                                                                    
tools=[ 
           WebSearchTool(), 
           function_tool(get_company_info),
           function_tool(get_eps_trend),
           function_tool(get_earnings_dates),
           function_tool(get_earnings_analysis),
           function_tool(get_historical_prices),
           function_tool(get_ticker_news),
           function_tool(search_news_by_ticker),
           function_tool(search_news_by_query),
           function_tool(search_companies),
           function_tool(get_top_value_companies),
           function_tool(get_top_growth_companies),
           ]                                                                                                                                                            
# Create enhanced agent with comprehensive instructions                                                                                                             
stock_agent_structured = Agent(                                                                                                                                                
    name="stock_analyst_agent",                                                                                                                                     
    tools=tools,  # Add web search to existing tools                                                                                                 
    model="gpt-5-mini",                                                                                                                                            
    instructions="""You are an expert stock market analyst with deep fundamental analysis expertise.                                                                
                                                                                                                                                                    
📊 TICKER REFERENCE GUIDE:                                                                                                                                          
Magnificent 7: AAPL (Apple), MSFT (Microsoft), GOOGL (Alphabet/Google), AMZN (Amazon), META (Meta/Facebook), TSLA (Tesla), NVDA (NVIDIA)                            
                                                                                                                                                                    
⚠️ CRITICAL TICKER CORRECTIONS:                                                                                                                                     
- FB → META (Facebook rebranded, FB ticker is deprecated)                                                                                                           
- GOOG/GOOGL → Use GOOGL for Class A shares                                                                                                                         
- If you get a 404 error, the ticker might be wrong:                                                                                                                
* Try alternate ticker (e.g., GOOG vs GOOGL)                                                                                                                      
* Use web_search to find correct ticker: "What is the stock ticker for [company]"                                                                                 
* Check if company was renamed or merged                                                                                                                          
                                                                                                                                                                    
🎯 ANALYSIS METHODOLOGY:                                                                                                                                            
When analyzing a company, follow this comprehensive approach:                                                                                                       
                                                                                                                                                                    
1. DATA COLLECTION (call tools for each):                                                                                                                           
    a) get_company_info(ticker) - Get fundamental data, valuation, price info                                                                                        
    b) get_eps_trend(ticker) - Get EPS trend data                                                                                                                    
    c) get_earnings_dates(ticker) - Get earnings history with surprises                                                                                              
    d) get_earnings_analysis(ticker) - Get analyst estimates and revisions                                                                                           
    e) get_historical_prices(ticker, period="6mo") - Get price history for momentum                                                                                  
    f) get_ticker_news(ticker, count=10) - Get recent company news                                                                                                   
    g) search_news_by_ticker(ticker, num_results=20) - Get comprehensive news                                                                                        
                                                                                                                                                                    
2. COMPETITIVE ANALYSIS:                                                                                                                                            
    - Identify 3-5 main competitors based on industry                                                                                                                
    - Call get_company_info for each competitor                                                                                                                      
    - Compare PE ratios, margins, growth rates                                                                                                                       
    - Use search_news_by_query to find competitor news                                                                                                               
                                                                                                                                                                    
3. TREND ANALYSIS:                                                                                                                                                  
    - EPS vs Market: Compare growth_estimates.stockTrend vs growth_estimates.indexTrend                                                                              
    - Analyst Sentiment: Check eps_revisions up/down counts (7d and 30d)                                                                                             
    - Price Position: Calculate distance from 52-week high/low                                                                                                       
    - Volume Trends: Analyze recent trading volume patterns                                                                                                          
                                                                                                                                                                    
4. NEWS SENTIMENT:                                                                                                                                                  
    - Analyze news titles and summaries for sentiment                                                                                                                
    - Identify key themes (e.g., "AI expansion", "regulatory issues", "new products")                                                                                
    - Look for catalyst events (earnings, product launches, partnerships)                                                                                            
    - Check competitor news for industry trends                                                                                                                      
                                                                                                                                                                    
5. VALUATION ASSESSMENT:                                                                                                                                            
    Compare these metrics:                                                                                                                                           
    - Current PE vs historical PE (if available)                                                                                                                     
    - Current PE vs sector/competitor average                                                                                                                        
    - Price vs 52-week range (cheap if <30% from low, expensive if >70% toward high)                                                                                 
    - Forward PE vs growth rate (PEG ratio concept)                                                                                                                  
                                                                                                                                                                    
6. DEMAND INDICATORS:                                                                                                                                               
    - Volume trend: Compare recent volume to average volume                                                                                                          
    - Price momentum: 3-month and 6-month returns                                                                                                                    
    - Trading activity: Check if volume is above/below average                                                                                                       
                                                                                                                                                                    
7. ERROR HANDLING:                                                                                                                                                  
    - If any tool returns 404 or error, use web_search to find correct information                                                                                   
    - Example: web_search("What is Meta stock ticker symbol")                                                                                                        
    - Always verify ticker before concluding failure                                                                                                                 
                                                                                                                                                                    
8. SYNTHESIS:                                                                                                                                                       
    - Provide clear "improving" vs "declining" assessments                                                                                                           
    - Give specific percentages and numbers                                                                                                                          
    - Flag red flags (declining EPS, negative revisions, weak volume)                                                                                                
    - Highlight green flags (beating estimates, positive revisions, strong momentum)                                                                                 
                                                                                                                                                                    
🔧 TOOL USAGE RULES:                                                                                                                                                
- Call tools sequentially, not in parallel, to ensure data consistency                                                                                              
- Don't skip tools - comprehensive analysis requires all data points                                                                                                
- If comparing companies, call tools for EACH company separately                                                                                                    
- Use web_search as backup when primary tools fail                                                                                                                  
- Always extract and use the correct ticker symbol                                                                                                                  
                                                                                                                                                                    
📈 OUTPUT FORMAT:                                                                                                                                                   
Structure your analysis to cover:                                                                                                                                   
1. EPS Trend vs Market (with specific numbers)                                                                                                                      
2. Analyst Sentiment (revision counts and direction)                                                                                                                
3. Recent News (company + competitors)                                                                                                                              
4. Demand Analysis (volume, momentum, activity level)                                                                                                               
5. Valuation (current vs 52-week range, PE comparison)                                                                                                              
6. Competitive Position (PE vs competitors)                                                                                                                         
7. Future Outlook (themes, catalysts, risks)                                                                                                                        
8. Executive Summary (concise investment thesis)                                                                                                                    
                                                                                                                                                                    
Always provide specific data points, percentages, and clear directional assessments.                                                                                
"""                                                                                                                                                            
)    

In [65]:
                                                                                                                                                                
import json                                                                                                                                                         
from datetime import datetime                                                                                                                                       
from typing import Optional                                                                                                                                         
                                                                                                                                                                    
async def analyze_company(                                                                                                                                          
    ticker: str,                                                                                                                                                    
    include_competitors: bool = True,                                                                                                                               
    context: Optional[RunResult] = None                                                                                                                             
) -> tuple[RunResult, dict]:                                                                                                                                        
    """                                                                                                                                                             
    Perform comprehensive company analysis.                                                                                                                         
                                                                                                                                                                    
    Args:                                                                                                                                                           
        ticker: Stock ticker symbol                                                                                                                                 
        include_competitors: Whether to analyze competitors (default True)                                                                                          
        context: Optional previous results for follow-up                                                                                                            
                                                                                                                                                                    
    Returns:                                                                                                                                                        
        (RunResult, structured_data_dict)                                                                                                                           
    """                                                                                                                                                             
                                                                                                                                                                    
    # Normalize ticker                                                                                                                                              
    ticker = ticker.upper()                                                                                                                                         
    if ticker == 'FB':                                                                                                                                              
        print("⚠️  Converting FB → META")                                                                                                                           
        ticker = 'META'                                                                                                                                             
                                                                                                                                                                    
    # Build comprehensive question                                                                                                                                  
    question = f"""Perform a comprehensive analysis of {ticker} covering:                                                                                           
                                                                                                                                                                    
1. EPS TREND ANALYSIS:                                                                                                                                              
    - Get current quarter EPS trend                                                                                                                                  
    - Compare to market index performance                                                                                                                            
    - Analyze surprise history (last 4 quarters)                                                                                                                     
    - Determine if trend is improving/declining                                                                                                                      
                                                                                                                                                                    
2. ANALYST SENTIMENT:                                                                                                                                               
    - Check EPS revision trends (7d and 30d)                                                                                                                         
    - Compare growth estimates to index                                                                                                                              
    - Determine if sentiment is improving/declining                                                                                                                  
                                                                                                                                                                    
3. RECENT NEWS:                                                                                                                                                     
    - Get recent company news (last 10 articles)                                                                                                                     
    - Search for competitor news                                                                                                                                     
    - Identify key themes and sentiment                                                                                                                              
    - Note any upcoming catalysts                                                                                                                                    
                                                                                                                                                                    
4. DEMAND ANALYSIS:                                                                                                                                                 
    - Analyze 3-month and 6-month price momentum                                                                                                                     
    - Check trading volume trends                                                                                                                                    
    - Determine trading activity level                                                                                                                               
                                                                                                                                                                    
5. VALUATION:                                                                                                                                                       
    - Current price vs 52-week high/low                                                                                                                              
    - Calculate distance from high (percentage)                                                                                                                      
    - Get PE ratio                                                                                                                                                   
    {f"- Compare PE to competitors' average" if include_competitors else ""}                                                                                         
                                                                                                                                                                    
{f'''6. COMPETITIVE COMPARISON:                                                                                                                                     
    - Identify 3-5 main competitors                                                                                                                                  
    - Get their PE ratios and market caps                                                                                                                            
    - Compare valuation metrics''' if include_competitors else ''}                                                                                                   
                                                                                                                                                                    
7. FUTURE OUTLOOK:                                                                                                                                                  
    - Identify key themes from news                                                                                                                                  
    - List upcoming catalyst events                                                                                                                                  
    - Note risk factors                                                                                                                                              
    - Provide sentiment direction                                                                                                                                    
                                                                                                                                                                    
Please use ALL available tools to gather complete data. If you encounter any errors, use web search to find correct information.                                    
                                                                                                                                                                    
Provide a structured analysis with specific numbers and clear assessments.                                                                                          
"""                                                                                                                                                                 
                                                                                                                                                                    
    print(f"\n{'='*70}")                                                                                                                                            
    print(f"📊 COMPREHENSIVE ANALYSIS: {ticker}")                                                                                                                   
    print('='*70)                                                                                                                                                   
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")                                                                                            
    print()                                                                                                                                                         
                                                                                                                                                                    
    # Run analysis                                                                                                                                                  
    if context:                                                                                                                                                     
        results = await runner.run(stock_agent, input=question, context=context.new_items)                                                                          
    else:                                                                                                                                                           
        results = await runner.run(stock_agent, input=question)                                                                                                     
                                                                                                                                                                    
    # Show tools called                                                                                                                                             
    print("\n" + "="*70)                                                                                                                                            
    print("🔧 TOOLS CALLED")                                                                                                                                        
    print("="*70)                                                                                                                                                   
                                                                                                                                                                    
    tool_calls = []                                                                                                                                                 
    for resp in results.raw_responses:                                                                                                                              
        for out in resp.output:                                                                                                                                     
            if hasattr(out, 'name'):                                                                                                                                
                tool_calls.append({                                                                                                                                 
                    'name': out.name,                                                                                                                               
                    'args': out.arguments                                                                                                                           
                })                                                                                                                                                  
                                                                                                                                                                    
    # Group by tool type                                                                                                                                            
    from collections import defaultdict                                                                                                                             
    tools_by_type = defaultdict(list)                                                                                                                               
    for call in tool_calls:                                                                                                                                         
        tools_by_type[call['name']].append(call['args'])                                                                                                            
                                                                                                                                                                    
    for tool_name, args_list in tools_by_type.items():                                                                                                              
        print(f"\n{tool_name}:")                                                                                                                                    
        for args in args_list:                                                                                                                                      
            print(f"  • {args}")                                                                                                                                    
                                                                                                                                                                    
    print(f"\nTotal tools called: {len(tool_calls)}")                                                                                                               
                                                                                                                                                                    
    # Extract structured data from results                                                                                                                          
    structured_data = extract_structured_data(results, ticker)                                                                                                      
                                                                                                                                                                    
    # Show summary                                                                                                                                                  
    print("\n" + "="*70)                                                                                                                                            
    print("📋 ANALYSIS SUMMARY")                                                                                                                                    
    print("="*70)                                                                                                                                                   
                                                                                                                                                                    
    if structured_data:                                                                                                                                             
        print(f"✓ EPS Trend: {structured_data.get('eps_trend', 'N/A')}")                                                                                            
        print(f"✓ Analyst Sentiment: {structured_data.get('analyst_sentiment', 'N/A')}")                                                                            
        print(f"✓ Valuation: {structured_data.get('valuation_summary', 'N/A')}")                                                                                    
        print(f"✓ News Items: {structured_data.get('news_count', 0)}")                                                                                              
        print(f"✓ Competitors Analyzed: {structured_data.get('competitor_count', 0)}")                                                                              
                                                                                                                                                                    
    # Show full response                                                                                                                                            
    print("\n" + "="*70)                                                                                                                                            
    print("💬 DETAILED ANALYSIS")                                                                                                                                   
    print("="*70)                                                                                                                                                   
    print(results.final_output)                                                                                                                                     
    print()                                                                                                                                                         
                                                                                                                                                                    
    return results, structured_data 

                                                                                                                                                           
                                                                                                                                                                    
def extract_structured_data(results: RunResult, ticker: str) -> dict:                                                                                               
    """Extract structured data from tool outputs."""                                                                                                                
                                                                                                                                                                    
    data = {                                                                                                                                                        
        'ticker': ticker,                                                                                                                                           
        'analysis_date': datetime.now().isoformat(),                                                                                                                
    }                                                                                                                                                               
                                                                                                                                                                    
    # Extract from tool outputs                                                                                                                                     
    for item in results.new_items:                                                                                                                                  
        if item.type != 'tool_call_output_item':                                                                                                                    
            continue                                                                                                                                                
                                                                                                                                                                    
        output = item.output                                                                                                                                        
        if not isinstance(output, dict):                                                                                                                            
            continue                                                                                                                                                
                                                                                                                                                                    
        # EPS trend data                                                                                                                                            
        if 'data' in output and isinstance(output['data'], list) and len(output['data']) > 0:                                                                       
            if 'current' in output['data'][0]:                                                                                                                      
                data['eps_trend'] = {                                                                                                                               
                    'current': output['data'][0].get('current'),                                                                                                    
                    'trend': 'declining' if output['data'][0].get('current', 0) < output['data'][0].get('90daysAgo', 0) else 'improving'                            
                }                                                                                                                                                   
                                                                                                                                                                    
        # Company info                                                                                                                                              
        if 'analyst' in output:                                                                                                                                     
            data['pe_ratio'] = output.get('valuation', {}).get('trailingPE')                                                                                        
            data['current_price'] = output.get('price', {}).get('currentPrice')                                                                                     
            data['week_52_high'] = output.get('price', {}).get('fiftyTwoWeekHigh')                                                                                  
            data['week_52_low'] = output.get('price', {}).get('fiftyTwoWeekLow')                                                                                    
                                                                                                                                                                    
            if data.get('current_price') and data.get('week_52_high'):                                                                                              
                distance = ((data['week_52_high'] - data['current_price']) / data['week_52_high']) * 100                                                            
                data['distance_from_high_pct'] = round(distance, 2)                                                                                                 
                                                                                                                                                                    
        # Earnings analysis                                                                                                                                         
        if 'earnings_estimates' in output:                                                                                                                          
            data['analyst_sentiment'] = analyze_revisions(output.get('eps_revisions', []))                                                                          
            data['growth_vs_index'] = output.get('growth_estimates', [{}])[0].get('stockTrend', 0)                                                                  
                                                                                                                                                                    
        # News                                                                                                                                                      
        if 'count' in output and isinstance(output.get('data'), list):                                                                                              
            data['news_count'] = len(output['data'])                                                                                                                
                                                                                                                                                                    
    # Add valuation summary                                                                                                                                         
    if 'distance_from_high_pct' in data:                                                                                                                            
        if data['distance_from_high_pct'] > 30:                                                                                                                     
            data['valuation_summary'] = 'cheap'                                                                                                                     
        elif data['distance_from_high_pct'] < 10:                                                                                                                   
            data['valuation_summary'] = 'expensive'                                                                                                                 
        else:                                                                                                                                                       
            data['valuation_summary'] = 'fair'                                                                                                                      
                                                                                                                                                                    
    return data                                                                                                                                                     
                                                                                                                                                                    
                                                                                                                                                                    
def analyze_revisions(eps_revisions: list) -> str:                                                                                                                  
    """Analyze EPS revision trends."""                                                                                                                              
    if not eps_revisions:                                                                                                                                           
        return "unknown"                                                                                                                                            
                                                                                                                                                                    
    recent = eps_revisions[0] if eps_revisions else {}                                                                                                              
    up_7d = recent.get('upLast7days', 0)                                                                                                                            
    down_7d = recent.get('downLast7Days', 0)                                                                                                                        
                                                                                                                                                                    
    if up_7d > down_7d:                                                                                                                                             
        return "improving"                                                                                                                                          
    elif down_7d > up_7d:                                                                                                                                           
        return "declining"                                                                                                                                          
    else:                                                                                                                                                           
        return "mixed"     

In [66]:
                                                                                                                                                                
# Example 1: Single company analysis                                                                                                                                
results, data = await analyze_company('TSLA')                                                                                                                       
                                                                                                                                                                    
# Access structured data                                                                                                                                            
print(f"\nEPS Trend: {data.get('eps_trend')}")                                                                                                                      
print(f"PE Ratio: {data.get('pe_ratio')}")                                                                                                                          
print(f"Distance from 52w high: {data.get('distance_from_high_pct')}%")                                                                                             
print(f"Valuation: {data.get('valuation_summary')}")                                                                                                                
                                                                                                                                                                    
# Example 2: Without competitor analysis (faster)                                                                                                                   
results, data = await analyze_company('NVDA', include_competitors=False)                                                                                            
                                                                                                                                                                    
# Example 3: Follow-up questions                                                                                                                                    
results1, data1 = await analyze_company('META')                                                                                                                     
results2 = await ask(stock_agent, "What are the biggest risks for META?", context=results1)                                                                         
                                                                                                                                                                    
# Example 4: Batch analysis                                                                                                                                         
companies = ['AAPL', 'MSFT', 'GOOGL']                                                                                                                               
analyses = {}                                                                                                                                                       
                                                                                                                                                                    
for ticker in companies:                                                                                                                                            
    print(f"\n\n{'#'*70}")                                                                                                                                          
    print(f"# ANALYZING {ticker}")                                                                                                                                  
    print('#'*70)                                                                                                                                                   
    results, data = await analyze_company(ticker, include_competitors=False)                                                                                        
    analyses[ticker] = data                                                                                                                                         
                                                                                                                                                                    
# Compare results                                                                                                                                                   
print("\n" + "="*70)                                                                                                                                                
print("COMPARATIVE SUMMARY")                                                                                                                                        
print("="*70)                                                                                                                                                       
for ticker, data in analyses.items():                                                                                                                               
    print(f"\n{ticker}:")                                                                                                                                           
    print(f"  PE Ratio: {data.get('pe_ratio', 'N/A')}")                                                                                                             
    print(f"  Distance from High: {data.get('distance_from_high_pct', 'N/A')}%")                                                                                    
    print(f"  Valuation: {data.get('valuation_summary', 'N/A')}")                                                                                                   
    print(f"  Analyst Sentiment: {data.get('analyst_sentiment', 'N/A')}")                                                                                           
                                                                                


📊 COMPREHENSIVE ANALYSIS: TSLA
Started at: 2026-01-07 20:56:39


🔧 TOOLS CALLED

get_eps_trend:
  • {"ticker":"TSLA"}

get_earnings_analysis:
  • {"ticker":"TSLA"}

get_ticker_news:
  • {"ticker":"TSLA","count":10}

get_historical_prices:
  • {"ticker":"TSLA","period":"6mo","interval":"1d"}

get_company_info:
  • {"ticker":"TSLA"}

get_earnings_dates:
  • {"ticker":"TSLA"}

Total tools called: 6

📋 ANALYSIS SUMMARY
✓ EPS Trend: {'current': 0.44048, 'trend': 'declining'}
✓ Analyst Sentiment: mixed
✓ Valuation: fair
✓ News Items: 10
✓ Competitors Analyzed: 0

💬 DETAILED ANALYSIS
Here's a comprehensive analysis of Tesla, Inc. (TSLA):

### 1. EPS Trend Analysis:
#### Current Quarter EPS Trend:
- **Current EPS Estimate:** $0.44048
- **7 Days Ago:** $0.44578
- **30 Days Ago:** $0.4488
- **60 Days Ago:** $0.43919
- **90 Days Ago:** $0.46771

#### Comparison to Market Index Performance:
- **EPS Growth for Current Quarter:** -39.66% compared to last year.
- **Market Index Trend:** 6.98% growth

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.



🔧 TOOLS CALLED

get_eps_trend:
  • {"ticker":"META"}

get_earnings_analysis:
  • {"ticker":"META"}

get_ticker_news:
  • {"ticker":"META","count":10}

get_historical_prices:
  • {"ticker":"META","period":"6mo","interval":"1d"}
  • {"ticker":"META","period":"3mo","interval":"1d"}

get_company_info:
  • {"ticker":"META"}

get_earnings_dates:
  • {"ticker":"META"}

Total tools called: 7

📋 ANALYSIS SUMMARY
✓ EPS Trend: {'current': 8.17046, 'trend': 'improving'}
✓ Analyst Sentiment: declining
✓ Valuation: fair
✓ News Items: 10
✓ Competitors Analyzed: 0

💬 DETAILED ANALYSIS
Here's a structured analysis of Meta Platforms, Inc. (META) based on the requested criteria:

### 1. EPS Trend Analysis
**Current Quarter EPS Trend:**
- Current EPS Estimate: **8.17**
- 7 Days Ago: **8.17**
- 30 Days Ago: **8.17**
- 60 Days Ago: **8.16**
- 90 Days Ago: **8.07**

**Comparison to Market Index Performance:**
- **EPS Growth Trend:**
  - Current quarter EPS growth: **1.88%**
  - Compared to market index grow

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.


### 💬 Response:

...


🔧 TOOLS CALLED

get_eps_trend:
  • {"ticker":"AAPL"}

get_earnings_analysis:
  • {"ticker":"AAPL"}

get_ticker_news:
  • {"ticker":"AAPL","count":10}

get_historical_prices:
  • {"ticker":"AAPL","period":"6mo","interval":"1d"}

get_company_info:
  • {"ticker":"AAPL"}

Total tools called: 5

📋 ANALYSIS SUMMARY
✓ EPS Trend: {'current': 2.66601, 'trend': 'improving'}
✓ Analyst Sentiment: improving
✓ Valuation: expensive
✓ News Items: 10
✓ Competitors Analyzed: 0

💬 DETAILED ANALYSIS
### Comprehensive Analysis of AAPL

#### 1. EPS Trend Analysis
- **Current Quarter EPS Trend**:
  - **Estimated EPS**: 2.66601 (current), 2.66721 (7 days ago), 2.66315 (30 days ago), and 2.48185 (90 days ago).
  - **Year-Over-Year Growth**: 10.11% (compared to 2.4 a year ago).
  
- **Market Index Performance**: 
  - **EPS Growth**: AAPL's current growth rate (11.08%) outperformed the broader market index’s trend of 6.98% in the same quarter.

- **Surprise History (last 4 quarters)**:
  - Q4: Actual EPS 2.4, E

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: max retries reached, giving up on this batch.



🔧 TOOLS CALLED

get_eps_trend:
  • {"ticker":"MSFT"}

get_earnings_analysis:
  • {"ticker":"MSFT"}

get_ticker_news:
  • {"ticker":"MSFT","count":10}

get_historical_prices:
  • {"ticker":"MSFT","period":"6mo","interval":"1d"}
  • {"ticker":"MSFT","period":"3mo","interval":"1d"}

get_company_info:
  • {"ticker":"MSFT"}

Total tools called: 6

📋 ANALYSIS SUMMARY
✓ EPS Trend: {'current': 3.86083, 'trend': 'improving'}
✓ Analyst Sentiment: declining
✓ Valuation: fair
✓ News Items: 10
✓ Competitors Analyzed: 0

💬 DETAILED ANALYSIS
**Comprehensive Analysis of Microsoft Corporation (MSFT)**

### 1. EPS TREND ANALYSIS:
#### Current Quarter EPS Trend
- **Current EPS**: 3.86083
- **7 Days Ago**: 3.86083
- **30 Days Ago**: 3.8575
- **60 Days Ago**: 3.86517
- **90 Days Ago**: 3.80502
- **Year-ago EPS**: 3.23, showing **growth of 19.53%**.

#### Comparison to Market Index Performance
- **Trend Improvement**: EPS shows consistency with slight fluctuations; generally stable trend indicating solid p

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.



🔧 TOOLS CALLED

get_eps_trend:
  • {"ticker":"GOOGL"}

get_earnings_analysis:
  • {"ticker":"GOOGL"}

get_ticker_news:
  • {"ticker":"GOOGL","count":10}

get_historical_prices:
  • {"ticker":"GOOGL","period":"6mo","interval":"1d"}
  • {"ticker":"GOOGL","period":"3mo","interval":"1d"}

get_company_info:
  • {"ticker":"GOOGL"}

Total tools called: 6

📋 ANALYSIS SUMMARY
✓ EPS Trend: {'current': 2.62911, 'trend': 'improving'}
✓ Analyst Sentiment: mixed
✓ Valuation: expensive
✓ News Items: 10
✓ Competitors Analyzed: 0

💬 DETAILED ANALYSIS
Here's a comprehensive analysis of **Alphabet Inc. (GOOGL)** based on the requested categories:

### 1. EPS Trend Analysis

- **Current Quarter EPS Trend:** 
  - **Avg EPS Estimate:** $2.62911
  - **EPS Estimates Over Time:** 
    - 7 days ago: $2.62689
    - 30 days ago: $2.62433
    - 60 days ago: $2.61779
    - 90 days ago: $2.54849

- **Surprise History (Last 4 Quarters):**
  1. **Q4 2023:** Actual: $2.15, Estimate: $2.125, Surprise: +0.02 (0.01%)
  2

In [67]:
# STRUCTURED OUTPUT OF ALL
analyses

{'AAPL': {'ticker': 'AAPL',
  'analysis_date': '2026-01-07T20:58:26.243482',
  'eps_trend': {'current': 2.66601, 'trend': 'improving'},
  'analyst_sentiment': 'improving',
  'growth_vs_index': 0.1109,
  'news_count': 10,
  'pe_ratio': None,
  'current_price': 260.11,
  'week_52_high': 288.62,
  'week_52_low': 169.21,
  'distance_from_high_pct': 9.88,
  'valuation_summary': 'expensive'},
 'MSFT': {'ticker': 'MSFT',
  'analysis_date': '2026-01-07T20:58:52.717292',
  'eps_trend': {'current': 3.86083, 'trend': 'improving'},
  'analyst_sentiment': 'declining',
  'growth_vs_index': 0.21040002,
  'news_count': 10,
  'pe_ratio': None,
  'current_price': 483.25,
  'week_52_high': 555.45,
  'week_52_low': 344.79,
  'distance_from_high_pct': 13.0,
  'valuation_summary': 'fair'},
 'GOOGL': {'ticker': 'GOOGL',
  'analysis_date': '2026-01-07T20:59:19.442802',
  'eps_trend': {'current': 2.62911, 'trend': 'improving'},
  'analyst_sentiment': 'mixed',
  'growth_vs_index': 0.22540002,
  'news_count': 10

In [68]:
analysis

{'ticker': 'TSLA',
 'earnings_estimates': [{'period': '0q',
   'avg': 0.44048,
   'low': 0.14,
   'high': 0.65,
   'yearAgoEps': 0.73,
   'numberOfAnalysts': 25,
   'growth': -0.3966},
  {'period': '+1q',
   'avg': 0.43644,
   'low': 0.27725,
   'high': 0.57,
   'yearAgoEps': 0.27,
   'numberOfAnalysts': 12,
   'growth': 0.6164},
  {'period': '0y',
   'avg': 1.63333,
   'low': 1.14846,
   'high': 2.03,
   'yearAgoEps': 2.42,
   'numberOfAnalysts': 34,
   'growth': -0.32509997},
  {'period': '+1y',
   'avg': 2.20383,
   'low': 1.31,
   'high': 3.48,
   'yearAgoEps': 1.63333,
   'numberOfAnalysts': 33,
   'growth': 0.3493}],
 'eps_revisions': [{'period': '0q',
   'upLast7days': 2,
   'upLast30days': 1,
   'downLast30days': 5,
   'downLast7Days': 2},
  {'period': '+1q',
   'upLast7days': 2,
   'upLast30days': 0,
   'downLast30days': 2,
   'downLast7Days': 0},
  {'period': '0y',
   'upLast7days': 3,
   'upLast30days': 2,
   'downLast30days': 6,
   'downLast7Days': 3},
  {'period': '+1y',
 

In [69]:
def print_analysis_report(data: dict):                                                                                                                              
      """Pretty print the analysis report."""                                                                                                                         
                                                                                                                                                                      
      print("\n" + "="*70)                                                                                                                                            
      print(f"📊 ANALYSIS REPORT: {data.get('ticker', 'N/A')}")                                                                                                       
      print("="*70)                                                                                                                                                   
                                                                                                                                                                      
      print(f"\n📅 Analysis Date: {data.get('analysis_date', 'N/A')}")                                                                                                
                                                                                                                                                                      
      print("\n" + "-"*70)                                                                                                                                            
      print("💹 EPS TREND VS. MARKET")                                                                                                                                
      print("-"*70)                                                                                                                                                   
      eps = data.get('eps_trend', {})                                                                                                                                 
      print(f"Current EPS: {eps.get('current', 'N/A')}")                                                                                                              
      print(f"Trend: {eps.get('trend', 'N/A')}")                                                                                                                      
                                                                                                                                                                      
      print("\n" + "-"*70)                                                                                                                                            
      print("👥 ANALYST SENTIMENT")                                                                                                                                   
      print("-"*70)                                                                                                                                                   
      print(f"Revision Trend: {data.get('analyst_sentiment', 'N/A')}")                                                                                                
      print(f"Growth vs Index: {data.get('growth_vs_index', 'N/A')}%")                                                                                                
                                                                                                                                                                      
      print("\n" + "-"*70)                                                                                                                                            
      print("💰 VALUATION")                                                                                                                                           
      print("-"*70)                                                                                                                                                   
      print(f"Current Price: ${data.get('current_price', 'N/A')}")                                                                                                    
      print(f"52-Week High: ${data.get('week_52_high', 'N/A')}")                                                                                                      
      print(f"52-Week Low: ${data.get('week_52_low', 'N/A')}")                                                                                                        
      print(f"Distance from High: {data.get('distance_from_high_pct', 'N/A')}%")                                                                                      
      print(f"PE Ratio: {data.get('pe_ratio', 'N/A')}")                                                                                                               
      print(f"Valuation: {data.get('valuation_summary', 'N/A').upper()}")                                                                                             
                                                                                                                                                                      
      print("\n" + "-"*70)                                                                                                                                            
      print("📰 NEWS & SENTIMENT")                                                                                                                                    
      print("-"*70)                                                                                                                                                   
      print(f"Articles Analyzed: {data.get('news_count', 'N/A')}")                                                                                                    
      print(f"Competitors Analyzed: {data.get('competitor_count', 0)}")                                                                                               
                                                                                                                                                                      
      print("\n" + "="*70)                                    

In [70]:
results, data = await analyze_company('AAPL')                                                                                                                       
print_analysis_report(data) 


📊 COMPREHENSIVE ANALYSIS: AAPL
Started at: 2026-01-07 20:59:19



[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: max retries reached, giving up on this batch.



🔧 TOOLS CALLED

get_eps_trend:
  • {"ticker":"AAPL"}

get_earnings_analysis:
  • {"ticker":"AAPL"}

get_ticker_news:
  • {"ticker":"AAPL","count":10}

get_historical_prices:
  • {"ticker":"AAPL","period":"6mo","interval":"1d"}

get_company_info:
  • {"ticker":"AAPL"}

get_earnings_dates:
  • {"ticker":"AAPL"}

Total tools called: 6

📋 ANALYSIS SUMMARY
✓ EPS Trend: {'current': 2.66601, 'trend': 'improving'}
✓ Analyst Sentiment: improving
✓ Valuation: expensive
✓ News Items: 10
✓ Competitors Analyzed: 0

💬 DETAILED ANALYSIS
### Comprehensive Analysis of Apple Inc. (AAPL)

---

#### 1. **EPS Trend Analysis**
- **Current Quarter EPS Trend**:
  - EPS Estimate: **$2.67**
  - EPS Surprises (Last 4 Quarters):
    - **Q1**: 2.4 (Est: 2.35) +2.26%
    - **Q2**: 1.65 (Est: 1.63) +1.50%
    - **Q3**: 1.57 (Est: 1.43) +9.48%
    - **Q4**: 1.85 (Est: 1.77) +4.52%
  
- **Historical EPS Trend**: Improved from **$2.48** to **$2.67**. Indicates a positive trend.
- **Comparison to Market Index Performan

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: max retries reached, giving up on this batch.
